In [13]:
git clone https://github.com/freqtrade/freqtrade.git

SyntaxError: invalid syntax (640821807.py, line 1)

In [1]:
# File: user_data/strategies/MyPandasTaStrategy.py

from freqtrade.strategy.interface import IStrategy
import pandas as pd
import pandas_ta as ta

class MyPandasTaStrategy(IStrategy):
    timeframe = '1h'
    startup_candle_count = 50  # Make sure we have enough data for all indicators

    # Indicator parameters -- you can optimize these later!
    adx_length = 14
    atr_length = 14
    ema_length = 20
    sma_length = 20
    rsi_length = 14
    cci_length = 20

    def populate_indicators(self, dataframe: pd.DataFrame, metadata: dict) -> pd.DataFrame:
        # ADX
        adx = ta.adx(dataframe['high'], dataframe['low'], dataframe['close'], length=self.adx_length)
        dataframe['ADX'] = adx[f'ADX_{self.adx_length}']
        dataframe['DMP'] = adx[f'DMP_{self.adx_length}']
        dataframe['DMN'] = adx[f'DMN_{self.adx_length}']

        # ATR
        dataframe['ATR'] = ta.atr(dataframe['high'], dataframe['low'], dataframe['close'], length=self.atr_length)

        # Bollinger Bands
        bbands = ta.bbands(dataframe['close'])
        dataframe = pd.concat([dataframe, bbands], axis=1)

        # CCI
        dataframe['CCI'] = ta.cci(dataframe['high'], dataframe['low'], dataframe['close'], length=self.cci_length)

        # EMA, SMA
        dataframe['EMA'] = ta.ema(dataframe['close'], length=self.ema_length)
        dataframe['SMA'] = ta.sma(dataframe['close'], length=self.sma_length)

        # MACD
        macd = ta.macd(dataframe['close'])
        dataframe = pd.concat([dataframe, macd], axis=1)

        # OBV
        dataframe['OBV'] = ta.obv(dataframe['close'], dataframe['volume'])

        # Parabolic SAR
        psar = ta.psar(dataframe['high'], dataframe['low'], dataframe['close'])
        dataframe['PSAR'] = psar['PSARl_0.02_0.2']

        # RSI
        dataframe['RSI'] = ta.rsi(dataframe['close'], length=self.rsi_length)

        # Stochastic Oscillator
        stoch = ta.stoch(dataframe['high'], dataframe['low'], dataframe['close'])
        dataframe = pd.concat([dataframe, stoch], axis=1)

        # Ichimoku
        ichi = ta.ichimoku(dataframe['high'], dataframe['low'], dataframe['close'])
        dataframe['ichimoku_base'] = ichi['ISA_9']

        # Fibonacci (not natively supported, often calculated for plotting not logic)
        # -- if you want fib levels, you'd custom-calculate them as needed
        # Example (not used in logic):

        return dataframe

    def populate_buy_trend(self, dataframe: pd.DataFrame, metadata: dict) -> pd.DataFrame:
        conditions = []

        # Uptrend and oversold composite condition
        conditions.append(dataframe['ADX'] > 25)
        conditions.append(dataframe['DMP'] > dataframe['DMN'])
        conditions.append(dataframe['RSI'] < 30)
        conditions.append(dataframe['CCI'] < -100)
        conditions.append(dataframe['close'] < dataframe['BBL_20_2.0'])
        conditions.append(dataframe['MACD_12_26_9'] > dataframe['MACDs_12_26_9'])
        conditions.append(dataframe['STOCHk_14_3_3'] < 20)
        conditions.append(dataframe['close'] > dataframe['EMA'])
        conditions.append(dataframe['close'] > dataframe['ichimoku_base'])
        conditions.append(dataframe['OBV'].diff() > 0)  # Rising OBV
        conditions.append(dataframe['PSAR'] < dataframe['close'])

        # All conditions must be true
        dataframe.loc[
            (dataframe[conditions].all(axis=1)),
            'buy'
        ] = 1

        return dataframe

    def populate_sell_trend(self, dataframe: pd.DataFrame, metadata: dict) -> pd.DataFrame:
        conditions = []

        # Downtrend and overbought composite condition
        conditions.append(dataframe['ADX'] > 25)
        conditions.append(dataframe['DMP'] < dataframe['DMN'])
        conditions.append(dataframe['RSI'] > 70)
        conditions.append(dataframe['CCI'] > 100)
        conditions.append(dataframe['close'] > dataframe['BBU_20_2.0'])
        conditions.append(dataframe['MACD_12_26_9'] < dataframe['MACDs_12_26_9'])
        conditions.append(dataframe['STOCHk_14_3_3'] > 80)
        conditions.append(dataframe['close'] < dataframe['EMA'])
        conditions.append(dataframe['close'] < dataframe['ichimoku_base'])
        conditions.append(dataframe['OBV'].diff() < 0)
        conditions.append(dataframe['PSAR'] > dataframe['close'])

        # All conditions must be true
        dataframe.loc[
            (dataframe[conditions].all(axis=1)),
            'sell'
        ] = 1

        return dataframe


ModuleNotFoundError: No module named 'freqtrade.strategy'

In [1]:
import sys
print(sys.executable)

/bin/python3.9


In [2]:
!{sys.executable} -m pip install freqtrade

Defaulting to user installation because normal site-packages is not writeable
  Using cached freqtrade-2024.9-py3-none-any.whl.metadata (18 kB)
  Using cached ccxt-4.4.96-py2.py3-none-any.whl.metadata (131 kB)
  Using cached humanize-4.12.3-py3-none-any.whl.metadata (7.8 kB)
  Using cached ta_lib-0.6.4.tar.gz (381 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached technical-1.5.0-py3-none-any.whl.metadata (7.9 kB)
  Using cached tabulate-0.9.0-py3-none-any.whl.metadata (34 kB)
  Using cached pycoingecko-3.2.0-py3-none-any.whl.metadata (16 kB)
  Using cached py_find_1st-1.1.7-cp39-cp39-linux_x86_64.whl
  Using cached python_rapidjson-1.21-cp39-cp39-manylinux_2_24_x86_64.manylinux_2_28_x86_64.whl.metadata (23 kB)
  Using cached questionary-2.1.0-py3-none-any.whl.metadata (5.4 kB)
  Using cached fastapi-0.116.1-py3-none-any.whl.metadata (28 kB)


In [3]:
pip install pandas ta

Defaulting to user installation because normal site-packages is not writeable
  Preparing metadata (setup.py) ... done
  DEPRECATION: Building 'ta' using the legacy setup.py bdist_wheel mechanism, which will be removed in a future version. pip 25.3 will enforce this behaviour change. A possible replacement is to use the standardized build interface by setting the `--use-pep517` option, (possibly combined with `--no-build-isolation`), or adding a `pyproject.toml` file to the source tree of 'ta'. Discussion can be found at https://github.com/pypa/pip/issues/6334
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=ba8a4f3de749a8a1cb4a04709fe4a71d9d27bec34476c14f04fb91dda99bdd0c
  Stored in directory: /home/justdial/.cache/pip/wheels/61/d8/66/8018676d483fa5edb5337a7a25ee8c029ac255be25f90f3cd5
Successfully built ta
Note: you may need to restart the kernel to use updated packages.


In [12]:
import yfinance as yf
import pandas as pd

df = yf.download("TSLA", start="2022-01-01", end="2024-07-20", interval="1d")
df = df.rename(columns={'Open':'open','High':'high','Low':'low','Close':'close','Volume':'volume'})
df = df[['open','high','low','close','volume']]
df.reset_index(inplace=True)
print(df.head())


Failed to get ticker 'TSLA' reason: Expecting value: line 1 column 1 (char 0)
[*********************100%***********************]  1 of 1 completed

1 Failed download:
['TSLA']: YFTzMissingError('$%ticker%: possibly delisted; no timezone found')


Empty DataFrame
Columns: [(Date, ), (open, TSLA), (high, TSLA), (low, TSLA), (close, TSLA), (volume, TSLA)]
Index: []


In [ ]:
import requests
import pandas as pd

# Welcome to Alpha Vantage! Your API key is: HTJTG8TQCYEVMZP6. Please record this API key at a safe place for future data access.

API_KEY = "HTJTG8TQCYEVMZP6"  # your provided API key
symbol = "TSLA"  # Example symbol, replace with your desired stock
url = f"https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol={symbol}&outputsize=full&apikey={API_KEY}"

r = requests.get(url)
data = r.json()

if "Time Series (Daily)" not in data:
    print("Problem fetching data:", data)
else:
    df = pd.DataFrame(data['Time Series (Daily)']).T
    df = df.rename(columns={
        "1. open": "open",
        "2. high": "high",
        "3. low": "low",
        "4. close": "close",
        "5. volume": "volume"
    })
    df = df.astype(float)
    df.index = pd.to_datetime(df.index)
    df = df.sort_index()
    print(df.head())


Problem fetching data: {}


In [14]:
df

,open,high,low,close,volume
2010-06-29,19.00,25.0000,17.54,23.89,18766300.0
2010-06-30,25.79,30.4192,23.30,23.83,17187100.0
2010-07-01,25.00,25.9200,20.27,21.96,8218800.0
2010-07-02,23.00,23.1000,18.71,19.20,5139800.0
2010-07-06,20.00,20.0000,15.83,16.11,6866900.0
...,...,...,...,...,...
2025-07-21,334.40,338.0000,326.88,328.49,75768797.0
2025-07-22,329.74,335.4098,321.55,332.11,77370371.0
2025-07-23,330.90,336.2000,328.67,332.56,92553756.0
2025-07-24,310.00,310.1500,300.41,305.30,156966023.0


In [1]:
import requests
import pandas as pd

API_KEY = "HTJTG8TQCYEVMZP6"  # your provided API key
symbol = "MSFT"               # Microsoft stock ticker
interval = "15min"

url = (
    f"https://www.alphavantage.co/query"
    f"?function=TIME_SERIES_INTRADAY"
    f"&symbol={symbol}"
    f"&interval={interval}"
    f"&outputsize=full"
    f"&apikey={API_KEY}"
)

r = requests.get(url)
data = r.json()

# Alpha Vantage returns the key like "Time Series (15min)"
timeseries_key = f"Time Series ({interval})"

if timeseries_key not in data:
    print("Problem fetching data:", data)
else:
    df = pd.DataFrame(data[timeseries_key]).T
    df = df.rename(columns={
        "1. open": "open",
        "2. high": "high",
        "3. low": "low",
        "4. close": "close",
        "5. volume": "volume"
    })
    df = df.astype(float)
    df.index = pd.to_datetime(df.index)
    df = df.sort_index()
    print(df.head())


                       open    high     low   close  volume
2025-06-30 04:00:00  497.40  498.39  497.34  497.80  5270.0
2025-06-30 04:15:00  497.79  497.95  497.40  497.50  2114.0
2025-06-30 04:30:00  497.58  497.75  497.36  497.73  1057.0
2025-06-30 04:45:00  497.61  497.94  497.58  497.58   933.0
2025-06-30 05:00:00  497.76  497.90  497.53  497.88  1001.0


In [4]:
df

,open,high,low,close,volume
2025-06-30 04:00:00,497.400,498.39,497.34,497.800,5270.0
2025-06-30 04:15:00,497.790,497.95,497.40,497.500,2114.0
2025-06-30 04:30:00,497.580,497.75,497.36,497.730,1057.0
2025-06-30 04:45:00,497.610,497.94,497.58,497.580,933.0
2025-06-30 05:00:00,497.760,497.90,497.53,497.880,1001.0
...,...,...,...,...,...
2025-07-29 18:45:00,513.860,514.38,513.40,514.085,2017.0
2025-07-29 19:00:00,514.085,514.35,513.52,514.250,1797.0
2025-07-29 19:15:00,514.300,514.32,514.00,514.010,4345.0
2025-07-29 19:30:00,514.190,514.31,513.77,513.920,3072.0


In [5]:
import pandas as pd
import ta  # Technical Analysis library

# Assuming `df` is your DataFrame with columns: open, high, low, close, volume

# Moving Average (44)
df["MA44"] = df["close"].rolling(window=44).mean()

# Bollinger Bands
df["bb_bbm"] = ta.volatility.BollingerBands(df["close"], window=20).bollinger_mavg()
df["bb_bbh"] = ta.volatility.BollingerBands(df["close"], window=20).bollinger_hband()
df["bb_bbl"] = ta.volatility.BollingerBands(df["close"], window=20).bollinger_lband()

# Ichimoku Cloud
df["ichimoku_a"] = ta.trend.IchimokuIndicator(df["high"], df["low"]).ichimoku_a()
df["ichimoku_b"] = ta.trend.IchimokuIndicator(df["high"], df["low"]).ichimoku_b()
df["ichimoku_base"] = ta.trend.IchimokuIndicator(df["high"], df["low"]).ichimoku_base_line()
df["ichimoku_conv"] = ta.trend.IchimokuIndicator(df["high"], df["low"]).ichimoku_conversion_line()


In [6]:
df

,open,high,low,close,volume,MA44,bb_bbm,bb_bbh,bb_bbl,ichimoku_a,ichimoku_b,ichimoku_base,ichimoku_conv
2025-06-30 04:00:00,497.400,498.39,497.34,497.800,5270.0,NaN,NaN,NaN,NaN,NaN,497.8650,NaN,NaN
2025-06-30 04:15:00,497.790,497.95,497.40,497.500,2114.0,NaN,NaN,NaN,NaN,NaN,497.8650,NaN,NaN
2025-06-30 04:30:00,497.580,497.75,497.36,497.730,1057.0,NaN,NaN,NaN,NaN,NaN,497.8650,NaN,NaN
2025-06-30 04:45:00,497.610,497.94,497.58,497.580,933.0,NaN,NaN,NaN,NaN,NaN,497.8650,NaN,NaN
2025-06-30 05:00:00,497.760,497.90,497.53,497.880,1001.0,NaN,NaN,NaN,NaN,NaN,497.8650,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-07-29 18:45:00,513.860,514.38,513.40,514.085,2017.0,513.360957,512.99396,514.341111,511.646809,503.27340,504.8884,503.2734,503.2734
2025-07-29 19:00:00,514.085,514.35,513.52,514.250,1797.0,513.374593,513.09196,514.503631,511.680289,504.24655,504.8884,503.2734,505.2197
2025-07-29 19:15:00,514.300,514.32,514.00,514.010,4345.0,513.382775,513.21296,514.498119,511.927801,508.54920,504.8884,503.2734,513.8250
2025-07-29 19:30:00,514.190,514.31,513.77,513.920,3072.0,513.395957,513.31346,514.484645,512.142275,508.54920,504.8884,503.2734,513.8250


In [ ]:
import pandas as pd
import numpy as np
import ta  # Make sure to install with: pip install ta


# =========================
# Step 0: Load Your Data
# =========================
import requests
import pandas as pd

API_KEY = "HTJTG8TQCYEVMZP6"  # your provided API key
symbol = "MSFT"               # Microsoft stock ticker
interval = "15min"

url = (
    f"https://www.alphavantage.co/query"
    f"?function=TIME_SERIES_INTRADAY"
    f"&symbol={symbol}"
    f"&interval={interval}"
    f"&outputsize=full"
    f"&apikey={API_KEY}"
)

r = requests.get(url)
data = r.json()

# Alpha Vantage returns the key like "Time Series (15min)"
timeseries_key = f"Time Series ({interval})"

if timeseries_key not in data:
    print("Problem fetching data:", data)
else:
    df = pd.DataFrame(data[timeseries_key]).T
    df = df.rename(columns={
        "1. open": "open",
        "2. high": "high",
        "3. low": "low",
        "4. close": "close",
        "5. volume": "volume"
    })
    df = df.astype(float)
    df.index = pd.to_datetime(df.index)
    df = df.sort_index()
    print(df.head())


# =========================
# Step 1: Calculate Indicators
# =========================


# Moving Average (44-period)
df["MA44"] = df["close"].rolling(window=44).mean()


# Bollinger Bands (20-period window, 2 std dev)
bb_indicator = ta.volatility.BollingerBands(close=df["close"], window=20, window_dev=2)
df["bb_bbm"] = bb_indicator.bollinger_mavg()
df["bb_bbh"] = bb_indicator.bollinger_hband()
df["bb_bbl"] = bb_indicator.bollinger_lband()


# Ichimoku Cloud indicator
ichimoku_indicator = ta.trend.IchimokuIndicator(high=df["high"], low=df["low"])
df["ichimoku_a"] = ichimoku_indicator.ichimoku_a()
df["ichimoku_b"] = ichimoku_indicator.ichimoku_b()
df["ichimoku_base"] = ichimoku_indicator.ichimoku_base_line()
df["ichimoku_conv"] = ichimoku_indicator.ichimoku_conversion_line()


# =========================
# Step 2: Clean Data (remove NaNs created by indicators)
# =========================
required_cols = [
    "MA44", "bb_bbm", "bb_bbh", "bb_bbl",
    "ichimoku_a", "ichimoku_b", "ichimoku_base", "ichimoku_conv",
    "open", "high", "low", "close", "volume"
]
df = df.dropna(subset=required_cols)


# =========================
# Step 3: Define Trading Signal Logic
# =========================


def get_signal(row):
    close = row["close"]
    ma44 = row["MA44"]
    ich_a = row["ichimoku_a"]
    ich_b = row["ichimoku_b"]
    bb_upper = row["bb_bbh"]
    bb_lower = row["bb_bbl"]


    # Buy Signal:
    # Price above MA44 and Ichimoku Cloud, and near/below lower Bollinger Band (rebounce)
    if (close > ma44) and (close > ich_a) and (close > ich_b) and (close <= bb_lower * 1.01):
        return "buy"


    # Sell Signal:
    # Price below MA44 and Ichimoku Cloud, and near/above upper Bollinger Band (overbought)
    elif (close < ma44) and (close < ich_a) and (close < ich_b) and (close >= bb_upper * 0.99):
        return "sell"


    # Otherwise Hold
    else:
        return "hold"


df["signal"] = df.apply(get_signal, axis=1)


# =========================
# Step 4: Volume Prediction (simple moving average of volume)
# =========================
df["volume_ma5"] = df["volume"].rolling(window=5).mean()
df["volume_prediction"] = df["volume_ma5"].shift(1)  # Predict next interval volume based on rolling average


# =========================
# Step 5: Backtest Paper Trading Simulation
# =========================
initial_cash = 100000  # Starting capital $100,000
cash = initial_cash
position = 0.0          # Number of shares currently held
position_entry_price = 0.0


trade_log = []


for idx, row in df.iterrows():
    signal = row["signal"]
    price = row["close"]


    # Enter long position when signal is 'buy' and no current position
    if signal == "buy" and position == 0:
        position = cash / price       # Buy as many shares as possible
        position_entry_price = price
        cash = 0                     # All cash invested
        trade_log.append({
            "date": idx,
            "trade_type": "buy",
            "price": price,
            "units": position,
            "cash": cash
        })
    
    # Exit long position when signal is 'sell' and currently holding shares
    elif signal == "sell" and position > 0:
        cash = position * price       # Sell all shares
        profit = cash - initial_cash
        profit_pct = (profit / initial_cash) * 100
        trade_log.append({
            "date": idx,
            "trade_type": "sell",
            "price": price,
            "units": -position,
            "cash": cash,
            "profit": profit,
            "profit_pct": profit_pct
        })
        position = 0
        position_entry_price = 0


    # Hold signal: no action, keep current position or no position


# Close any open position at the last available price
if position > 0:
    last_price = df["close"].iloc[-1]
    cash = position * last_price
    profit = cash - initial_cash
    profit_pct = (profit / initial_cash) * 100
    trade_log.append({
        "date": df.index[-1],
        "trade_type": "sell",
        "price": last_price,
        "units": -position,
        "cash": cash,
        "profit": profit,
        "profit_pct": profit_pct
    })
    position = 0
    position_entry_price = 0


trade_df = pd.DataFrame(trade_log).set_index("date")


# =========================
# Add position_type classification to Trade Log
# =========================

def classify_position(trade_type):
    """
    Classify position as 'long' or 'short'.
    Current logic supports only long trades.
    """
    if trade_type in ['buy', 'sell']:
        return 'long'
    else:
        return 'none'

trade_df["position_type"] = trade_df["trade_type"].apply(classify_position)


# =========================
# Step 6: Calculate Profit Summaries by Timeframe
# =========================


def profit_summary(trades, freq, initial_capital=100000):
    sells = trades[trades["trade_type"] == "sell"]
    if sells.empty:
        return pd.DataFrame()  
    grouped = sells.resample(freq).agg({"profit": "sum"})
    grouped["profit_pct"] = (grouped["profit"] / initial_capital) * 100
    grouped = grouped.rename(columns={"profit": "total_profit"})
    return grouped


daily_profit = profit_summary(trade_df, "D")
weekly_profit = profit_summary(trade_df, "W")
monthly_profit = profit_summary(trade_df, "M")


# =========================
# Step 7: Output Results with Explanation
# =========================


print("\n===== Trade Log =====")
print(trade_df)


print("\n===== Daily Profit Summary =====")
print(daily_profit)


print("\n===== Weekly Profit Summary =====")
print(weekly_profit)


print("\n===== Monthly Profit Summary =====")
print(monthly_profit)



# =========================
# Summary Explanation and Recommendations
# =========================


total_profit = monthly_profit["total_profit"].sum()
total_profit_pct = (total_profit / initial_cash) * 100


print("\n" + "="*40)
print(f"Initial Capital: ${initial_cash:,.2f}")
print(f"Total Profit over period (Monthly summed): ${total_profit:,.2f}")
print(f"Total Profit Percentage: {total_profit_pct:.2f}%")


# Determine best timeframe by profit_pct
best_timeframe = "daily"
best_profit_pct = daily_profit["profit_pct"].sum()


if weekly_profit["profit_pct"].sum() > best_profit_pct:
    best_timeframe = "weekly"
    best_profit_pct = weekly_profit["profit_pct"].sum()


if monthly_profit["profit_pct"].sum() > best_profit_pct:
    best_timeframe = "monthly"
    best_profit_pct = monthly_profit["profit_pct"].sum()


print(f"Best performing timeframe according to backtest: {best_timeframe.capitalize()} "
      f"(Total Profit %: {best_profit_pct:.2f}%)")


# Trading recommendation based on last known signal and position (from trade log)
last_signal = df["signal"].iloc[-1]


if last_signal == "buy":
    recommendation = "Based on the latest indicator signals, it is advisable to GO LONG (BUY)."
elif last_signal == "sell":
    recommendation = ("Based on the latest signals, consider SELLING your position or STAY OUT "
                      "(note: short selling not implemented).")
else:
    recommendation = "No clear buy/sell signal at present. Hold your current position."


print("\nTrading Recommendation:")
print(recommendation)


print("="*40)


# Optional: To save results to CSV uncomment below lines
# trade_df.to_csv("trade_log.csv")
# daily_profit.to_csv("daily_profit.csv")
# weekly_profit.to_csv("weekly_profit.csv")
# monthly_profit.to_csv("monthly_profit.csv")


                       open    high     low   close  volume
2025-06-30 04:00:00  497.40  498.39  497.34  497.80  5270.0
2025-06-30 04:15:00  497.79  497.95  497.40  497.50  2114.0
2025-06-30 04:30:00  497.58  497.75  497.36  497.73  1057.0
2025-06-30 04:45:00  497.61  497.94  497.58  497.58   933.0
2025-06-30 05:00:00  497.76  497.90  497.53  497.88  1001.0

===== Trade Log =====
                    trade_type     price       units           cash  \
date                                                                  
2025-06-30 14:45:00        buy  498.7775  200.490199       0.000000   
2025-06-30 15:45:00       sell  497.1600 -200.490199   99675.707104   
2025-07-02 09:30:00        buy  493.0900  202.145059       0.000000   
2025-07-02 14:00:00       sell  490.2300 -202.145059   99097.572235   
2025-07-02 16:30:00        buy  491.3050  201.702755       0.000000   
2025-07-07 09:15:00       sell  497.6555 -201.702755  100378.485583   
2025-07-08 16:00:00        buy  496.7600  202.066

/tmp/ipykernel_127560/3864855980.py:223: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  grouped = sells.resample(freq).agg({"profit": "sum"})


In [15]:
import pandas as pd
import numpy as np
import ta  # Make sure to install with: pip install ta

# =========================
# Step 0: Load Your Data
# =========================
# Replace/make sure you have your DataFrame 'df' loaded with columns:
# open, high, low, close, volume
# For example:
# df = pd.read_csv("your_data.csv", parse_dates=True, index_col=0)

# =========================
# Step 1: Calculate Indicators
# =========================

# Moving Average (44-period)
df["MA44"] = df["close"].rolling(window=44).mean()

# Bollinger Bands (20-period window, 2 std dev)
bb_indicator = ta.volatility.BollingerBands(close=df["close"], window=20, window_dev=2)
df["bb_bbm"] = bb_indicator.bollinger_mavg()
df["bb_bbh"] = bb_indicator.bollinger_hband()
df["bb_bbl"] = bb_indicator.bollinger_lband()

# Ichimoku Cloud indicator
ichimoku_indicator = ta.trend.IchimokuIndicator(high=df["high"], low=df["low"])
df["ichimoku_a"] = ichimoku_indicator.ichimoku_a()
df["ichimoku_b"] = ichimoku_indicator.ichimoku_b()
df["ichimoku_base"] = ichimoku_indicator.ichimoku_base_line()
df["ichimoku_conv"] = ichimoku_indicator.ichimoku_conversion_line()

# =========================
# Step 2: Clean Data (remove NaNs created by indicators)
# =========================
required_cols = [
    "MA44", "bb_bbm", "bb_bbh", "bb_bbl",
    "ichimoku_a", "ichimoku_b", "ichimoku_base", "ichimoku_conv",
    "open", "high", "low", "close", "volume"
]
df = df.dropna(subset=required_cols)

# =========================
# Step 3: Define Trading Signal Logic
# =========================

def get_signal(row):
    close = row["close"]
    ma44 = row["MA44"]
    ich_a = row["ichimoku_a"]
    ich_b = row["ichimoku_b"]
    bb_upper = row["bb_bbh"]
    bb_lower = row["bb_bbl"]

    # Buy Signal:
    # Price above MA44 and Ichimoku Cloud, and near/below lower Bollinger Band (rebounce)
    if (close > ma44) and (close > ich_a) and (close > ich_b) and (close <= bb_lower * 1.01):
        return "buy"

    # Sell Signal:
    # Price below MA44 and Ichimoku Cloud, and near/above upper Bollinger Band (overbought)
    elif (close < ma44) and (close < ich_a) and (close < ich_b) and (close >= bb_upper * 0.99):
        return "sell"

    # Otherwise Hold
    else:
        return "hold"

df["signal"] = df.apply(get_signal, axis=1)

# =========================
# Step 4: Volume Prediction (simple moving average of volume)
# =========================
df["volume_ma5"] = df["volume"].rolling(window=5).mean()
df["volume_prediction"] = df["volume_ma5"].shift(1)  # Predict next interval volume based on rolling average

# =========================
# Step 5: Backtest Paper Trading Simulation
# =========================
initial_cash = 100000  # Starting capital $100,000
cash = initial_cash
position = 0.0          # Number of shares currently held
position_entry_price = 0.0

trade_log = []

for idx, row in df.iterrows():
    signal = row["signal"]
    price = row["close"]

    # Enter long position when signal is 'buy' and no current position
    if signal == "buy" and position == 0:
        position = cash / price       # Buy as many shares as possible
        position_entry_price = price
        cash = 0                     # All cash invested
        trade_log.append({
            "date": idx,
            "trade_type": "buy",
            "price": price,
            "units": position,
            "cash": cash
        })
    
    # Exit long position when signal is 'sell' and currently holding shares
    elif signal == "sell" and position > 0:
        cash = position * price       # Sell all shares
        profit = cash - initial_cash
        profit_pct = (profit / initial_cash) * 100
        trade_log.append({
            "date": idx,
            "trade_type": "sell",
            "price": price,
            "units": -position,
            "cash": cash,
            "profit": profit,
            "profit_pct": profit_pct
        })
        position = 0
        position_entry_price = 0

    # Hold signal: no action, keep current position or no position

# Close any open position at the last available price
if position > 0:
    last_price = df["close"].iloc[-1]
    cash = position * last_price
    profit = cash - initial_cash
    profit_pct = (profit / initial_cash) * 100
    trade_log.append({
        "date": df.index[-1],
        "trade_type": "sell",
        "price": last_price,
        "units": -position,
        "cash": cash,
        "profit": profit,
        "profit_pct": profit_pct
    })
    position = 0
    position_entry_price = 0

trade_df = pd.DataFrame(trade_log).set_index("date")

# =========================
# Step 6: Calculate Profit Summaries by Timeframe
# =========================

def profit_summary(trades, freq, initial_capital=100000):
    sells = trades[trades["trade_type"] == "sell"]
    if sells.empty:
        return pd.DataFrame()  
    grouped = sells.resample(freq).agg({"profit": "sum"})
    grouped["profit_pct"] = (grouped["profit"] / initial_capital) * 100
    grouped = grouped.rename(columns={"profit": "total_profit"})
    return grouped

daily_profit = profit_summary(trade_df, "D")
weekly_profit = profit_summary(trade_df, "W")
monthly_profit = profit_summary(trade_df, "M")

# =========================
# Step 7: Output Results with Explanation
# =========================

print("\n===== Trade Log =====")
print(trade_df)

print("\n===== Daily Profit Summary =====")
print(daily_profit)

print("\n===== Weekly Profit Summary =====")
print(weekly_profit)

print("\n===== Monthly Profit Summary =====")
print(monthly_profit)


# =========================
# Summary Explanation and Recommendations
# =========================

initial_capital = 100000

total_profit = monthly_profit["total_profit"].sum()
total_profit_pct = (total_profit / initial_capital) * 100

print("\n" + "="*40)
print(f"Initial Capital: ${initial_capital:,.2f}")
print(f"Total Profit over period (Monthly summed): ${total_profit:,.2f}")
print(f"Total Profit Percentage: {total_profit_pct:.2f}%")

# Determine best timeframe by profit_pct
best_timeframe = "daily"
best_profit_pct = daily_profit["profit_pct"].sum()

if weekly_profit["profit_pct"].sum() > best_profit_pct:
    best_timeframe = "weekly"
    best_profit_pct = weekly_profit["profit_pct"].sum()

if monthly_profit["profit_pct"].sum() > best_profit_pct:
    best_timeframe = "monthly"
    best_profit_pct = monthly_profit["profit_pct"].sum()

print(f"Best performing timeframe according to backtest: {best_timeframe.capitalize()} "
      f"(Total Profit %: {best_profit_pct:.2f}%)")

# Trading recommendation based on last known signal and position (from trade log)
last_signal = df["signal"].iloc[-1]

if last_signal == "buy":
    recommendation = "Based on the latest indicator signals, it is advisable to GO LONG (BUY)."
elif last_signal == "sell":
    recommendation = ("Based on the latest signals, consider SELLING your position or STAY OUT "
                      "(note: short selling not implemented).")
else:
    recommendation = "No clear buy/sell signal at present. Hold your current position."

print("\nTrading Recommendation:")
print(recommendation)

print("="*40)


# Optional: To save results to CSV uncomment below lines
# trade_df.to_csv("trade_log.csv")
# daily_profit.to_csv("daily_profit.csv")
# weekly_profit.to_csv("weekly_profit.csv")
# monthly_profit.to_csv("monthly_profit.csv")


===== Trade Log =====
                    trade_type     price       units           cash  \
date                                                                  
2025-07-02 09:30:00        buy  493.0900  202.802734       0.000000   
2025-07-02 14:00:00       sell  490.2300 -202.802734   99419.984181   
2025-07-02 16:30:00        buy  491.3050  202.358991       0.000000   
2025-07-07 09:15:00       sell  497.6555 -202.358991  100705.064955   
2025-07-08 16:00:00        buy  496.7600  202.723780       0.000000   
2025-07-15 04:45:00       sell  502.0100 -202.723780  101769.364800   
2025-07-15 09:15:00        buy  503.0200  202.316737       0.000000   
2025-07-16 18:00:00       sell  504.8200 -202.316737  102133.534926   
2025-07-17 14:00:00        buy  512.0800  199.448397       0.000000   
2025-07-18 13:15:00       sell  510.9050 -199.448397  101899.183060   
2025-07-21 14:15:00        buy  510.5000  199.606627       0.000000   
2025-07-21 16:15:00       sell  508.5000 -199.606627  

/tmp/ipykernel_127560/1735107949.py:151: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  grouped = sells.resample(freq).agg({"profit": "sum"})


In [8]:
!{sys.executable} -m pip install freqtrade pandas ta

Defaulting to user installation because normal site-packages is not writeable


### GPT

In [ ]:
pip install ta pandas requests

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [18]:
import pandas as pd
import numpy as np
import requests
import time
import ta

# Configuration
API_KEY = "HTJTG8TQCYEVMZP6"
SYMBOL = "MSFT"
INTERVAL = "15min"
INITIAL_CAPITAL = 100000
STOP_LOSS_PCT = 0.03
TAKE_PROFIT_PCT = 0.06
SLIPPAGE_PCT = 0.001
COMMISSION_PCT = 0.001


def fetch_data(symbol, interval, api_key):
    url = (
        f"https://www.alphavantage.co/query"
        f"?function=TIME_SERIES_INTRADAY"
        f"&symbol={symbol}"
        f"&interval={interval}"
        f"&outputsize=full"
        f"&apikey={api_key}"
    )
    for _ in range(5):
        r = requests.get(url)
        data = r.json()
        key = f"Time Series ({interval})"
        if key in data:
            df = pd.DataFrame(data[key]).T.rename(columns={
                "1. open": "open",
                "2. high": "high",
                "3. low": "low",
                "4. close": "close",
                "5. volume": "volume"
            }).astype(float)
            df.index = pd.to_datetime(df.index)
            return df.sort_index()
        time.sleep(60)
    raise ValueError("Failed to fetch data after 5 retries.")


def compute_indicators(df):
    df["MA44"] = df["close"].rolling(window=44).mean()
    bb = ta.volatility.BollingerBands(df["close"], window=20, window_dev=2)
    df["bb_bbm"] = bb.bollinger_mavg()
    df["bb_bbh"] = bb.bollinger_hband()
    df["bb_bbl"] = bb.bollinger_lband()
    ich = ta.trend.IchimokuIndicator(df["high"], df["low"])
    df["ichimoku_a"] = ich.ichimoku_a()
    df["ichimoku_b"] = ich.ichimoku_b()
    df["ichimoku_base"] = ich.ichimoku_base_line()
    df["ichimoku_conv"] = ich.ichimoku_conversion_line()
    df["rsi"] = ta.momentum.RSIIndicator(df["close"]).rsi()
    df = df.dropna()
    return df


def generate_signals(df):
    def signal_logic(row):
        if row["ichimoku_conv"] > row["ichimoku_base"] and row["close"] > row["ichimoku_a"] and row["rsi"] < 30:
            return "buy"
        elif row["ichimoku_conv"] < row["ichimoku_base"] and row["close"] < row["ichimoku_b"] and row["rsi"] > 70:
            return "sell"
        return "hold"
    df["signal"] = df.apply(signal_logic, axis=1)
    return df


def backtest(df):
    cash = INITIAL_CAPITAL
    position = 0
    entry_price = 0
    trade_log = []
    for idx, row in df.iterrows():
        signal, price = row["signal"], row["close"]
        price_with_slippage = price * (1 + SLIPPAGE_PCT)

        if signal == "buy" and position == 0:
            position = (cash * (1 - COMMISSION_PCT)) / price_with_slippage
            entry_price = price_with_slippage
            cash = 0
            trade_log.append({"date": idx, "type": "buy", "price": price, "units": position, "cash": 0})

        elif position > 0:
            current_return = (price - entry_price) / entry_price
            if signal == "sell" or current_return <= -STOP_LOSS_PCT or current_return >= TAKE_PROFIT_PCT:
                exit_price = price * (1 - SLIPPAGE_PCT)
                cash = position * exit_price * (1 - COMMISSION_PCT)
                profit = cash - INITIAL_CAPITAL
                trade_log.append({
                    "date": idx, "type": "sell", "price": price, "units": -position,
                    "cash": cash, "profit": profit, "return_pct": (profit / INITIAL_CAPITAL) * 100
                })
                position = 0
                entry_price = 0

    if position > 0:
        final_price = df["close"].iloc[-1]
        exit_price = final_price * (1 - SLIPPAGE_PCT)
        cash = position * exit_price * (1 - COMMISSION_PCT)
        profit = cash - INITIAL_CAPITAL
        trade_log.append({
            "date": df.index[-1], "type": "sell", "price": final_price, "units": -position,
            "cash": cash, "profit": profit, "return_pct": (profit / INITIAL_CAPITAL) * 100
        })

    return pd.DataFrame(trade_log).set_index("date")


def evaluate(trades):
    if "profit" not in trades.columns:
        print("No trades executed.")
        return

    trades["cumulative_profit"] = trades["profit"].cumsum()
    win_rate = (trades["profit"] > 0).mean()
    avg_return = trades["return_pct"].mean()
    max_drawdown = trades["cumulative_profit"].cummax() - trades["cumulative_profit"]
    result = {
        "Total Trades": len(trades) // 2,
        "Win Rate (%)": round(win_rate * 100, 2),
        "Avg Return (%)": round(avg_return, 2),
        "Total Profit ($)": round(trades["profit"].sum(), 2),
        "Max Drawdown ($)": round(max_drawdown.max(), 2)
    }
    return result, trades


# Run
df = fetch_data(SYMBOL, INTERVAL, API_KEY)
df = compute_indicators(df)
df = generate_signals(df)
trades = backtest(df)
metrics, trade_log = evaluate(trades)

print("\n===== Trading Metrics =====")
for k, v in metrics.items():
    print(f"{k}: {v}")

print("\n===== Trade Log (first 5 entries) =====")
print(trade_log.head())



===== Trading Metrics =====
Total Trades: 2
Win Rate (%): 25.0
Avg Return (%): -5.5
Total Profit ($): -11001.29
Max Drawdown ($): 12579.37

===== Trade Log (first 5 entries) =====
                     type    price       units           cash        profit  \
date                                                                          
2025-07-02 06:30:00   buy  490.100  203.632320       0.000000           NaN   
2025-07-07 06:15:00  sell  499.830 -203.632320  101578.080966   1578.080966   
2025-07-11 04:30:00   buy  499.710  202.867919       0.000000           NaN   
2025-07-25 16:45:00  sell  431.787 -202.867919   87420.626237 -12579.373763   

                     return_pct  cumulative_profit  
date                                                
2025-07-02 06:30:00         NaN                NaN  
2025-07-07 06:15:00    1.578081        1578.080966  
2025-07-11 04:30:00         NaN                NaN  
2025-07-25 16:45:00  -12.579374      -11001.292797  


In [20]:
pip install pandas numpy ta matplotlib requests


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [25]:
pip install plotly


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [39]:
import pandas as pd
import numpy as np
import requests
from ta.trend import IchimokuIndicator
from ta.volatility import BollingerBands
import matplotlib.pyplot as plt
import seaborn as sns


# -----------------------------------
# Step 0: Load Intraday Data
# -----------------------------------
API_KEY = "HTJTG8TQCYEVMZP6"
symbol = "MSFT"
interval = "15min"

url = (
    f"https://www.alphavantage.co/query"
    f"?function=TIME_SERIES_INTRADAY"
    f"&symbol={symbol}"
    f"&interval={interval}"
    f"&outputsize=full"
    f"&apikey={API_KEY}"
)

response = requests.get(url)
data = response.json()
key = f"Time Series ({interval})"

if key not in data:
    raise ValueError("Error fetching data:", data)

df = pd.DataFrame(data[key]).T
df = df.rename(columns={
    "1. open": "open",
    "2. high": "high",
    "3. low": "low",
    "4. close": "close",
    "5. volume": "volume"
})
df = df.astype(float)
df.index = pd.to_datetime(df.index)
df = df.sort_index()

# -----------------------------------
# Step 1: Calculate Technical Indicators
# -----------------------------------
df["MA44"] = df["close"].rolling(window=44).mean()

bb = BollingerBands(close=df["close"], window=20, window_dev=2)
df["bb_bbm"] = bb.bollinger_mavg()
df["bb_bbh"] = bb.bollinger_hband()
df["bb_bbl"] = bb.bollinger_lband()

ich = IchimokuIndicator(high=df["high"], low=df["low"])
df["ichimoku_a"] = ich.ichimoku_a()
df["ichimoku_b"] = ich.ichimoku_b()
df["ichimoku_base"] = ich.ichimoku_base_line()
df["ichimoku_conv"] = ich.ichimoku_conversion_line()

# -----------------------------------
# Step 2: Clean Data
# -----------------------------------
df = df.dropna(subset=[
    "MA44", "bb_bbm", "bb_bbh", "bb_bbl",
    "ichimoku_a", "ichimoku_b", "ichimoku_base", "ichimoku_conv"
])

# -----------------------------------
# Step 3: Signal Logic
# -----------------------------------
def get_signal(row):
    close = row["close"]
    ma = row["MA44"]
    ich_a = row["ichimoku_a"]
    ich_b = row["ichimoku_b"]
    bb_upper = row["bb_bbh"]
    bb_lower = row["bb_bbl"]

    if close > ma and close > ich_a and close > ich_b and close <= bb_lower * 1.01:
        return "buy"
    elif close < ma and close < ich_a and close < ich_b and close >= bb_upper * 0.99:
        return "sell"
    else:
        return "hold"

df["signal"] = df.apply(get_signal, axis=1)

# -----------------------------------
# Step 4: Volume Prediction
# -----------------------------------
df["volume_ma5"] = df["volume"].rolling(window=5).mean()
df["volume_prediction"] = df["volume_ma5"].shift(1)

# -----------------------------------
# Step 5: Backtest Logic
# -----------------------------------
initial_cash = 100000
cash = initial_cash
position = 0
entry_price = 0
log = []

for idx, row in df.iterrows():
    signal = row["signal"]
    price = row["close"]

    if signal == "buy" and position == 0:
        position = cash / price
        entry_price = price
        cash = 0
        log.append({"date": idx, "trade_type": "buy", "price": price, "units": position, "cash": cash})

    elif signal == "sell" and position > 0:
        cash = position * price
        profit = cash - initial_cash
        log.append({
            "date": idx,
            "trade_type": "sell",
            "price": price,
            "units": -position,
            "cash": cash,
            "profit": profit,
            "profit_pct": (profit / initial_cash) * 100
        })
        position = 0
        entry_price = 0

# Final sell if holding
if position > 0:
    last_price = df["close"].iloc[-1]
    cash = position * last_price
    profit = cash - initial_cash
    log.append({
        "date": df.index[-1],
        "trade_type": "sell",
        "price": last_price,
        "units": -position,
        "cash": cash,
        "profit": profit,
        "profit_pct": (profit / initial_cash) * 100
    })
    position = 0

trade_df = pd.DataFrame(log).set_index("date")

# -----------------------------------
# Step 6: Profit Summaries
# -----------------------------------
def profit_summary(trades, freq):
    sells = trades[trades["trade_type"] == "sell"]
    if sells.empty:
        return pd.DataFrame()
    grouped = sells.resample(freq).agg({"profit": "sum"})
    grouped["profit_pct"] = (grouped["profit"] / initial_cash) * 100
    return grouped.rename(columns={"profit": "total_profit"})

daily_profit = profit_summary(trade_df, "D")
weekly_profit = profit_summary(trade_df, "W")
monthly_profit = profit_summary(trade_df, "M")

# -----------------------------------
# Step 7: Final Report
# -----------------------------------
print("\n===== Trade Log =====")
print(trade_df)
print("\n===== Daily Profit Summary =====")
print(daily_profit)
print("\n===== Weekly Profit Summary =====")
print(weekly_profit)
print("\n===== Monthly Profit Summary =====")
print(monthly_profit)

total_profit = monthly_profit["total_profit"].sum()
total_profit_pct = (total_profit / initial_cash) * 100

print("\n" + "="*40)
print(f"Initial Capital: ${initial_cash:,.2f}")
print(f"Total Profit (Monthly summed): ${total_profit:,.2f}")
print(f"Total Profit %: {total_profit_pct:.2f}%")

best_timeframe = max(
    [("daily", daily_profit), ("weekly", weekly_profit), ("monthly", monthly_profit)],
    key=lambda x: x[1]["profit_pct"].sum() if not x[1].empty else -np.inf
)[0]

print(f"Best Performing Timeframe: {best_timeframe.capitalize()}")

last_signal = df["signal"].iloc[-1]
if last_signal == "buy":
    recommendation = "Recommendation: GO LONG (BUY)"
elif last_signal == "sell":
    recommendation = "Recommendation: SELL or STAY OUT"
else:
    recommendation = "Recommendation: HOLD (no clear signal)"
print(recommendation)
print("="*40)


# -----------------------------------
# Step 8: MACD and RSI Calculation
# -----------------------------------
# MACD
exp1 = df['close'].ewm(span=12, adjust=False).mean()
exp2 = df['close'].ewm(span=26, adjust=False).mean()
df['MACD'] = exp1 - exp2
df['Signal_Line'] = df['MACD'].ewm(span=9, adjust=False).mean()

# RSI
delta = df['close'].diff()
gain = delta.where(delta > 0, 0)
loss = -delta.where(delta < 0, 0)
avg_gain = gain.rolling(window=14).mean()
avg_loss = loss.rolling(window=14).mean()
rs = avg_gain / avg_loss
df['RSI'] = 100 - (100 / (1 + rs))

# Drop rows with NaN values for indicators
plot_df = df.dropna(subset=["MACD", "Signal_Line", "RSI"])

# -----------------------------------
# Step 9: Plotting
# -----------------------------------
import plotly.graph_objs as go
from plotly.subplots import make_subplots

# Filter the data
plot_df = df.dropna(subset=[
    "close", "MA44", "bb_bbm", "bb_bbh", "bb_bbl",
    "ichimoku_a", "ichimoku_b", "MACD", "Signal_Line", "RSI", "signal"
])

buy_df = plot_df[plot_df["signal"] == "buy"]
sell_df = plot_df[plot_df["signal"] == "sell"]

# Create the subplots
fig = make_subplots(
    rows=3, cols=1, shared_xaxes=True,
    subplot_titles=("Price with Ichimoku, MA44 & Bollinger Bands", "MACD", "RSI"),
    row_heights=[0.5, 0.25, 0.25],
    vertical_spacing=0.05
)

# --- Chart 1: Price, Ichimoku, MA44, Bollinger ---
fig.add_trace(go.Scatter(x=plot_df.index, y=plot_df['ichimoku_a'], name='Ichimoku A', line=dict(color='green', width=1)), row=1, col=1)
fig.add_trace(go.Scatter(
    x=plot_df.index, y=plot_df['ichimoku_b'], name='Ichimoku B',
    line=dict(color='red', width=1), fill='tonexty',
    fillcolor='rgba(100, 100, 255, 0.2)'
), row=1, col=1)

fig.add_trace(go.Scatter(x=plot_df.index, y=plot_df['close'], name='Close', line=dict(color='white')), row=1, col=1)
fig.add_trace(go.Scatter(x=plot_df.index, y=plot_df['MA44'], name='MA44', line=dict(color='orange', dash='dash')), row=1, col=1)
fig.add_trace(go.Scatter(x=plot_df.index, y=plot_df['bb_bbh'], name='BB Upper', line=dict(color='blue', width=1)), row=1, col=1)
fig.add_trace(go.Scatter(x=plot_df.index, y=plot_df['bb_bbm'], name='BB Middle', line=dict(color='blue', dash='dot')), row=1, col=1)
fig.add_trace(go.Scatter(x=plot_df.index, y=plot_df['bb_bbl'], name='BB Lower', line=dict(color='blue', width=1)), row=1, col=1)

# Buy/Sell markers
fig.add_trace(go.Scatter(x=buy_df.index, y=buy_df['close'], mode='markers', name='Buy',
                         marker=dict(color='lime', symbol='triangle-up', size=10)), row=1, col=1)
fig.add_trace(go.Scatter(x=sell_df.index, y=sell_df['close'], mode='markers', name='Sell',
                         marker=dict(color='red', symbol='triangle-down', size=10)), row=1, col=1)

# --- Chart 2: MACD ---
fig.add_trace(go.Scatter(x=plot_df.index, y=plot_df['MACD'], name='MACD', line=dict(color='green')), row=2, col=1)
fig.add_trace(go.Scatter(x=plot_df.index, y=plot_df['Signal_Line'], name='Signal Line', line=dict(color='red')), row=2, col=1)

# --- Chart 3: RSI ---
fig.add_trace(go.Scatter(x=plot_df.index, y=plot_df['RSI'], name='RSI', line=dict(color='purple')), row=3, col=1)

# Add RSI Overbought and Oversold levels
fig.add_shape(type="line", x0=plot_df.index[0], x1=plot_df.index[-1], y0=70, y1=70,
              line=dict(color="red", dash="dash"), xref='x3', yref='y3')
fig.add_shape(type="line", x0=plot_df.index[0], x1=plot_df.index[-1], y0=30, y1=30,
              line=dict(color="green", dash="dash"), xref='x3', yref='y3')

# Layout settings
fig.update_layout(
    height=2000,
    title="Interactive Technical Analysis Dashboard (MSFT)",
    xaxis_rangeslider_visible=False,
    legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1),
    template="plotly_dark",
    hovermode="x unified"
)

fig.show()

import plotly.graph_objs as go
from plotly.subplots import make_subplots

# Prepare filtered data for charting
plot_df = df.dropna(subset=["close", "MA44", "bb_bbm", "bb_bbh", "bb_bbl", "ichimoku_a", "ichimoku_b", "signal"])

# --------- Bollinger Bands Chart ---------
fig_bb = go.Figure()
fig_bb.add_trace(go.Scatter(x=plot_df.index, y=plot_df['close'], name='Close', line=dict(color='white')))
fig_bb.add_trace(go.Scatter(x=plot_df.index, y=plot_df['bb_bbh'], name='BB Upper', line=dict(color='blue')))
fig_bb.add_trace(go.Scatter(x=plot_df.index, y=plot_df['bb_bbm'], name='BB Middle', line=dict(color='blue', dash='dot')))
fig_bb.add_trace(go.Scatter(x=plot_df.index, y=plot_df['bb_bbl'], name='BB Lower', line=dict(color='blue')))
fig_bb.update_layout(title='Bollinger Bands with Close Price', template='plotly_dark')
fig_bb.show()

# --------- Ichimoku Cloud Chart ---------
fig_ich = go.Figure()
fig_ich.add_trace(go.Scatter(x=plot_df.index, y=plot_df['ichimoku_a'], name='Ichimoku A', line=dict(color='green')))
fig_ich.add_trace(go.Scatter(
    x=plot_df.index, y=plot_df['ichimoku_b'], name='Ichimoku B',
    line=dict(color='red'), fill='tonexty', fillcolor='rgba(100, 100, 255, 0.2)'
))
fig_ich.add_trace(go.Scatter(x=plot_df.index, y=plot_df['close'], name='Close', line=dict(color='white')))
fig_ich.update_layout(title='Ichimoku Cloud with Close Price', template='plotly_dark')
fig_ich.show()

# --------- 44-period Moving Average Chart ---------
fig_ma44 = go.Figure()
fig_ma44.add_trace(go.Scatter(x=plot_df.index, y=plot_df['close'], name='Close', line=dict(color='white')))
fig_ma44.add_trace(go.Scatter(x=plot_df.index, y=plot_df['MA44'], name='MA44', line=dict(color='orange', dash='dash')))
fig_ma44.update_layout(title='44 Period Moving Average with Close Price', template='plotly_dark')
fig_ma44.show()

# --------- Combined Chart ---------
fig_combined = make_subplots(rows=1, cols=1, shared_xaxes=True, subplot_titles=('Price with Ichimoku, MA44 & Bollinger Bands',))
fig_combined.add_trace(go.Scatter(x=plot_df.index, y=plot_df['ichimoku_a'], name='Ichimoku A', line=dict(color='green')))
fig_combined.add_trace(go.Scatter(
    x=plot_df.index, y=plot_df['ichimoku_b'], name='Ichimoku B',
    line=dict(color='red'), fill='tonexty', fillcolor='rgba(100, 100, 255, 0.2)'
))
fig_combined.add_trace(go.Scatter(x=plot_df.index, y=plot_df['MA44'], name='MA44', line=dict(color='orange', dash='dash')))
fig_combined.add_trace(go.Scatter(x=plot_df.index, y=plot_df['bb_bbh'], name='BB Upper', line=dict(color='blue')))
fig_combined.add_trace(go.Scatter(x=plot_df.index, y=plot_df['bb_bbm'], name='BB Middle', line=dict(color='blue', dash='dot')))
fig_combined.add_trace(go.Scatter(x=plot_df.index, y=plot_df['bb_bbl'], name='BB Lower', line=dict(color='blue')))
fig_combined.add_trace(go.Scatter(x=plot_df.index, y=plot_df['close'], name='Close', line=dict(color='white')))
fig_combined.update_layout(title='Combined Price Chart with Ichimoku Cloud, MA44 and Bollinger Bands', template='plotly_dark')
fig_combined.show()



===== Trade Log =====
                    trade_type     price       units           cash  \
date                                                                  
2025-06-30 14:45:00        buy  498.7775  200.490199       0.000000   
2025-06-30 15:45:00       sell  497.1600 -200.490199   99675.707104   
2025-07-02 09:30:00        buy  493.0900  202.145059       0.000000   
2025-07-02 14:00:00       sell  490.2300 -202.145059   99097.572235   
2025-07-02 16:30:00        buy  491.3050  201.702755       0.000000   
2025-07-07 09:15:00       sell  497.6555 -201.702755  100378.485583   
2025-07-08 16:00:00        buy  496.7600  202.066361       0.000000   
2025-07-15 04:45:00       sell  502.0100 -202.066361  101439.333980   
2025-07-15 09:15:00        buy  503.0200  201.660638       0.000000   
2025-07-16 18:00:00       sell  504.8200 -201.660638  101802.323128   
2025-07-17 14:00:00        buy  512.0800  198.801600       0.000000   
2025-07-18 13:15:00       sell  510.9050 -198.801600  

/tmp/ipykernel_127560/2343635219.py:154: FutureWarning:

'M' is deprecated and will be removed in a future version, please use 'ME' instead.



In [32]:
import pandas as pd
import numpy as np
import requests
from ta.trend import IchimokuIndicator
from ta.volatility import BollingerBands
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objs as go
from plotly.subplots import make_subplots

# -----------------------------------
# Step 0: Load Intraday Data
# -----------------------------------
API_KEY = "HTJTG8TQCYEVMZP6"
symbol = "MSFT"
interval = "15min"

url = (
    f"https://www.alphavantage.co/query"
    f"?function=TIME_SERIES_INTRADAY"
    f"&symbol={symbol}"
    f"&interval={interval}"
    f"&outputsize=full"
    f"&apikey={API_KEY}"
)

response = requests.get(url)
data = response.json()
key = f"Time Series ({interval})"

if key not in data:
    raise ValueError("Error fetching data:", data)

df = pd.DataFrame(data[key]).T
df = df.rename(columns={
    "1. open": "open",
    "2. high": "high",
    "3. low": "low",
    "4. close": "close",
    "5. volume": "volume"
})
df = df.astype(float)
df.index = pd.to_datetime(df.index)
df = df.sort_index()

# -----------------------------------
# Step 1: Calculate Technical Indicators
# -----------------------------------
df["MA44"] = df["close"].rolling(window=44).mean()

bb = BollingerBands(close=df["close"], window=20, window_dev=2)
df["bb_bbm"] = bb.bollinger_mavg()
df["bb_bbh"] = bb.bollinger_hband()
df["bb_bbl"] = bb.bollinger_lband()

ich = IchimokuIndicator(high=df["high"], low=df["low"])
df["ichimoku_a"] = ich.ichimoku_a()
df["ichimoku_b"] = ich.ichimoku_b()
df["ichimoku_base"] = ich.ichimoku_base_line()
df["ichimoku_conv"] = ich.ichimoku_conversion_line()

# -----------------------------------
# Step 2: Clean Data
# -----------------------------------
df = df.dropna(subset=[
    "MA44", "bb_bbm", "bb_bbh", "bb_bbl",
    "ichimoku_a", "ichimoku_b", "ichimoku_base", "ichimoku_conv"
])

# -----------------------------------
# Step 3: Signal Logic + Reasoning Function
# -----------------------------------
def get_signal(row):
    close = row["close"]
    ma = row["MA44"]
    ich_a = row["ichimoku_a"]
    ich_b = row["ichimoku_b"]
    bb_upper = row["bb_bbh"]
    bb_lower = row["bb_bbl"]

    if close > ma and close > ich_a and close > ich_b and close <= bb_lower * 1.01:
        return "buy"
    elif close < ma and close < ich_a and close < ich_b and close >= bb_upper * 0.99:
        return "sell"
    else:
        return "hold"

def signal_reason(row):
    """Return string explanation of which indicators triggered the current signal."""
    close = row["close"]
    ma = row["MA44"]
    ich_a = row["ichimoku_a"]
    ich_b = row["ichimoku_b"]
    bb_upper = row["bb_bbh"]
    bb_lower = row["bb_bbl"]
    messages = []

    # MA44 comparison
    if close > ma:
        messages.append(f"Close ({close:.2f}) > MA44 ({ma:.2f})")
    else:
        messages.append(f"Close ({close:.2f}) <= MA44 ({ma:.2f})")

    # Ichimoku span comparisons
    if close > ich_a:
        messages.append(f"Close ({close:.2f}) > Ichimoku A ({ich_a:.2f})")
    else:
        messages.append(f"Close ({close:.2f}) <= Ichimoku A ({ich_a:.2f})")

    if close > ich_b:
        messages.append(f"Close ({close:.2f}) > Ichimoku B ({ich_b:.2f})")
    else:
        messages.append(f"Close ({close:.2f}) <= Ichimoku B ({ich_b:.2f})")

    # Bollinger Band proximity
    buy_band = close <= bb_lower * 1.01
    sell_band = close >= bb_upper * 0.99

    if buy_band:
        messages.append(f"Close ({close:.2f}) <= Lower BB * 1.01 ({bb_lower * 1.01:.2f})")
    if sell_band:
        messages.append(f"Close ({close:.2f}) >= Upper BB * 0.99 ({bb_upper * 0.99:.2f})")

    return "; ".join(messages)

df["signal"] = df.apply(get_signal, axis=1)

# -----------------------------------
# Step 4: Volume Prediction
# -----------------------------------
df["volume_ma5"] = df["volume"].rolling(window=5).mean()
df["volume_prediction"] = df["volume_ma5"].shift(1)

# -----------------------------------
# Step 5: Backtest Logic with Reasoning in Log
# -----------------------------------
initial_cash = 100000
cash = initial_cash
position = 0
entry_price = 0
log = []

for idx, row in df.iterrows():
    signal = row["signal"]
    price = row["close"]
    reasoning = signal_reason(row)

    if signal == "buy" and position == 0:
        position = cash / price
        entry_price = price
        cash = 0
        log.append({
            "date": idx,
            "trade_type": "buy",
            "price": price,
            "units": position,
            "cash": cash,
            "signal": signal,
            "reason": reasoning,
            "position_type": "long"
        })

    elif signal == "sell" and position > 0:
        cash = position * price
        profit = cash - initial_cash
        log.append({
            "date": idx,
            "trade_type": "sell",
            "price": price,
            "units": -position,
            "cash": cash,
            "profit": profit,
            "profit_pct": (profit / initial_cash) * 100,
            "signal": signal,
            "reason": reasoning,
            "position_type": "long"
        })
        position = 0
        entry_price = 0

# Final sell if still holding
if position > 0:
    last_price = df["close"].iloc[-1]
    cash = position * last_price
    profit = cash - initial_cash
    log.append({
        "date": df.index[-1],
        "trade_type": "sell",
        "price": last_price,
        "units": -position,
        "cash": cash,
        "profit": profit,
        "profit_pct": (profit / initial_cash) * 100,
        "signal": df["signal"].iloc[-1],
        "reason": signal_reason(df.iloc[-1]),
        "position_type": "long"
    })
    position = 0

trade_df = pd.DataFrame(log).set_index("date")

# -----------------------------------
# Step 6: Profit Summaries
# -----------------------------------
def profit_summary(trades, freq):
    sells = trades[trades["trade_type"] == "sell"]
    if sells.empty:
        return pd.DataFrame()
    grouped = sells.resample(freq).agg({"profit": "sum"})
    grouped["profit_pct"] = (grouped["profit"] / initial_cash) * 100
    return grouped.rename(columns={"profit": "total_profit"})

daily_profit = profit_summary(trade_df, "D")
weekly_profit = profit_summary(trade_df, "W")
monthly_profit = profit_summary(trade_df, "M")


# -----------------------------------
# Step 7: Final Report & Summed Totals
# -----------------------------------
print("\n===== Trade Log with Reasoning =====")
print(trade_df[[
    "trade_type", "price", "units", "cash", "profit", "profit_pct", "signal", "reason", "position_type"
]])

print("\n===== Daily Profit Summary =====")
print(daily_profit)
print(f"Total Daily Profit: {daily_profit['total_profit'].sum():,.2f}")

print("\n===== Weekly Profit Summary =====")
print(weekly_profit)
print(f"Total Weekly Profit: {weekly_profit['total_profit'].sum():,.2f}")

print("\n===== Monthly Profit Summary =====")
print(monthly_profit)
print(f"Total Monthly Profit: {monthly_profit['total_profit'].sum():,.2f}")

total_profit = monthly_profit["total_profit"].sum()
total_profit_pct = (total_profit / initial_cash) * 100

print("\n" + "="*40)
print(f"Initial Capital: ${initial_cash:,.2f}")
print(f"Total Profit (Monthly summed): ${total_profit:,.2f}")
print(f"Total Profit %: {total_profit_pct:.2f}%")

best_timeframe = max(
    [("daily", daily_profit), ("weekly", weekly_profit), ("monthly", monthly_profit)],
    key=lambda x: x[1]["profit_pct"].sum() if not x[1].empty else -np.inf
)[0]

print(f"Best Performing Timeframe: {best_timeframe.capitalize()}")

last_signal = df["signal"].iloc[-1]
if last_signal == "buy":
    recommendation = "Recommendation: GO LONG (BUY)"
elif last_signal == "sell":
    recommendation = "Recommendation: SELL or STAY OUT"
else:
    recommendation = "Recommendation: HOLD (no clear signal)"

print(recommendation)
print("="*40)

# -----------------------------------
# Step 8: MACD and RSI Calculation (optional for further analysis)
# -----------------------------------
# MACD
exp1 = df['close'].ewm(span=12, adjust=False).mean()
exp2 = df['close'].ewm(span=26, adjust=False).mean()
df['MACD'] = exp1 - exp2
df['Signal_Line'] = df['MACD'].ewm(span=9, adjust=False).mean()

# RSI
delta = df['close'].diff()
gain = delta.where(delta > 0, 0)
loss = -delta.where(delta < 0, 0)
avg_gain = gain.rolling(window=14).mean()
avg_loss = loss.rolling(window=14).mean()
rs = avg_gain / avg_loss
df['RSI'] = 100 - (100 / (1 + rs))


# -----------------------------------
# Step 9: Plotting
# -----------------------------------
import plotly.graph_objs as go
from plotly.subplots import make_subplots

# Filter the data
plot_df = df.dropna(subset=[
    "close", "MA44", "bb_bbm", "bb_bbh", "bb_bbl",
    "ichimoku_a", "ichimoku_b", "MACD", "Signal_Line", "RSI", "signal"
])

buy_df = plot_df[plot_df["signal"] == "buy"]
sell_df = plot_df[plot_df["signal"] == "sell"]

# Create the subplots
fig = make_subplots(
    rows=3, cols=1, shared_xaxes=True,
    subplot_titles=("Price with Ichimoku, MA44 & Bollinger Bands", "MACD", "RSI"),
    row_heights=[0.5, 0.25, 0.25],
    vertical_spacing=0.05
)

# --- Chart 1: Price, Ichimoku, MA44, Bollinger ---
fig.add_trace(go.Scatter(x=plot_df.index, y=plot_df['ichimoku_a'], name='Ichimoku A', line=dict(color='green', width=1)), row=1, col=1)
fig.add_trace(go.Scatter(
    x=plot_df.index, y=plot_df['ichimoku_b'], name='Ichimoku B',
    line=dict(color='red', width=1), fill='tonexty',
    fillcolor='rgba(100, 100, 255, 0.2)'
), row=1, col=1)

fig.add_trace(go.Scatter(x=plot_df.index, y=plot_df['close'], name='Close', line=dict(color='white')), row=1, col=1)
fig.add_trace(go.Scatter(x=plot_df.index, y=plot_df['MA44'], name='MA44', line=dict(color='orange', dash='dash')), row=1, col=1)
fig.add_trace(go.Scatter(x=plot_df.index, y=plot_df['bb_bbh'], name='BB Upper', line=dict(color='blue', width=1)), row=1, col=1)
fig.add_trace(go.Scatter(x=plot_df.index, y=plot_df['bb_bbm'], name='BB Middle', line=dict(color='blue', dash='dot')), row=1, col=1)
fig.add_trace(go.Scatter(x=plot_df.index, y=plot_df['bb_bbl'], name='BB Lower', line=dict(color='blue', width=1)), row=1, col=1)

# Buy/Sell markers
fig.add_trace(go.Scatter(x=buy_df.index, y=buy_df['close'], mode='markers', name='Buy',
                         marker=dict(color='lime', symbol='triangle-up', size=10)), row=1, col=1)
fig.add_trace(go.Scatter(x=sell_df.index, y=sell_df['close'], mode='markers', name='Sell',
                         marker=dict(color='red', symbol='triangle-down', size=10)), row=1, col=1)

# --- Chart 2: MACD ---
fig.add_trace(go.Scatter(x=plot_df.index, y=plot_df['MACD'], name='MACD', line=dict(color='green')), row=2, col=1)
fig.add_trace(go.Scatter(x=plot_df.index, y=plot_df['Signal_Line'], name='Signal Line', line=dict(color='red')), row=2, col=1)

# --- Chart 3: RSI ---
fig.add_trace(go.Scatter(x=plot_df.index, y=plot_df['RSI'], name='RSI', line=dict(color='purple')), row=3, col=1)

# Add RSI Overbought and Oversold levels
fig.add_shape(type="line", x0=plot_df.index[0], x1=plot_df.index[-1], y0=70, y1=70,
              line=dict(color="red", dash="dash"), xref='x3', yref='y3')
fig.add_shape(type="line", x0=plot_df.index[0], x1=plot_df.index[-1], y0=30, y1=30,
              line=dict(color="green", dash="dash"), xref='x3', yref='y3')

# Layout settings
fig.update_layout(
    height=900,
    title="Interactive Technical Analysis Dashboard (MSFT)",
    xaxis_rangeslider_visible=False,
    legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1),
    template="plotly_dark",
    hovermode="x unified"
)

fig.show()

import plotly.graph_objs as go
from plotly.subplots import make_subplots

# Prepare filtered data for charting
plot_df = df.dropna(subset=["close", "MA44", "bb_bbm", "bb_bbh", "bb_bbl", "ichimoku_a", "ichimoku_b", "signal"])

# --------- Bollinger Bands Chart ---------
fig_bb = go.Figure()
fig_bb.add_trace(go.Scatter(x=plot_df.index, y=plot_df['close'], name='Close', line=dict(color='white')))
fig_bb.add_trace(go.Scatter(x=plot_df.index, y=plot_df['bb_bbh'], name='BB Upper', line=dict(color='blue')))
fig_bb.add_trace(go.Scatter(x=plot_df.index, y=plot_df['bb_bbm'], name='BB Middle', line=dict(color='blue', dash='dot')))
fig_bb.add_trace(go.Scatter(x=plot_df.index, y=plot_df['bb_bbl'], name='BB Lower', line=dict(color='blue')))
fig_bb.update_layout(title='Bollinger Bands with Close Price', template='plotly_dark')
fig_bb.show()

# --------- Ichimoku Cloud Chart ---------
fig_ich = go.Figure()
fig_ich.add_trace(go.Scatter(x=plot_df.index, y=plot_df['ichimoku_a'], name='Ichimoku A', line=dict(color='green')))
fig_ich.add_trace(go.Scatter(
    x=plot_df.index, y=plot_df['ichimoku_b'], name='Ichimoku B',
    line=dict(color='red'), fill='tonexty', fillcolor='rgba(100, 100, 255, 0.2)'
))
fig_ich.add_trace(go.Scatter(x=plot_df.index, y=plot_df['close'], name='Close', line=dict(color='white')))
fig_ich.update_layout(title='Ichimoku Cloud with Close Price', template='plotly_dark')
fig_ich.show()

# --------- 44-period Moving Average Chart ---------
fig_ma44 = go.Figure()
fig_ma44.add_trace(go.Scatter(x=plot_df.index, y=plot_df['close'], name='Close', line=dict(color='white')))
fig_ma44.add_trace(go.Scatter(x=plot_df.index, y=plot_df['MA44'], name='MA44', line=dict(color='orange', dash='dash')))
fig_ma44.update_layout(title='44 Period Moving Average with Close Price', template='plotly_dark')
fig_ma44.show()

# --------- Combined Chart ---------
fig_combined = make_subplots(rows=1, cols=1, shared_xaxes=True, subplot_titles=('Price with Ichimoku, MA44 & Bollinger Bands',))
fig_combined.add_trace(go.Scatter(x=plot_df.index, y=plot_df['ichimoku_a'], name='Ichimoku A', line=dict(color='green')))
fig_combined.add_trace(go.Scatter(
    x=plot_df.index, y=plot_df['ichimoku_b'], name='Ichimoku B',
    line=dict(color='red'), fill='tonexty', fillcolor='rgba(100, 100, 255, 0.2)'
))
fig_combined.add_trace(go.Scatter(x=plot_df.index, y=plot_df['MA44'], name='MA44', line=dict(color='orange', dash='dash')))
fig_combined.add_trace(go.Scatter(x=plot_df.index, y=plot_df['bb_bbh'], name='BB Upper', line=dict(color='blue')))
fig_combined.add_trace(go.Scatter(x=plot_df.index, y=plot_df['bb_bbm'], name='BB Middle', line=dict(color='blue', dash='dot')))
fig_combined.add_trace(go.Scatter(x=plot_df.index, y=plot_df['bb_bbl'], name='BB Lower', line=dict(color='blue')))
fig_combined.add_trace(go.Scatter(x=plot_df.index, y=plot_df['close'], name='Close', line=dict(color='white')))
fig_combined.update_layout(title='Combined Price Chart with Ichimoku Cloud, MA44 and Bollinger Bands', template='plotly_dark')
fig_combined.show()



===== Trade Log with Reasoning =====
                    trade_type     price       units           cash  \
date                                                                  
2025-06-30 14:45:00        buy  498.7775  200.490199       0.000000   
2025-06-30 15:45:00       sell  497.1600 -200.490199   99675.707104   
2025-07-02 09:30:00        buy  493.0900  202.145059       0.000000   
2025-07-02 14:00:00       sell  490.2300 -202.145059   99097.572235   
2025-07-02 16:30:00        buy  491.3050  201.702755       0.000000   
2025-07-07 09:15:00       sell  497.6555 -201.702755  100378.485583   
2025-07-08 16:00:00        buy  496.7600  202.066361       0.000000   
2025-07-15 04:45:00       sell  502.0100 -202.066361  101439.333980   
2025-07-15 09:15:00        buy  503.0200  201.660638       0.000000   
2025-07-16 18:00:00       sell  504.8200 -201.660638  101802.323128   
2025-07-17 14:00:00        buy  512.0800  198.801600       0.000000   
2025-07-18 13:15:00       sell  510.905

/tmp/ipykernel_127560/1666552227.py:209: FutureWarning:

'M' is deprecated and will be removed in a future version, please use 'ME' instead.



In [38]:
# Optional: To save results to CSV uncomment below lines
# trade_df.to_csv("trade_log.csv")
daily_profit.to_csv("daily_profit_summary.csv")
weekly_profit.to_csv("weekly_profit_summary.csv")
monthly_profit.to_csv("monthly_profit_summary.csv")
# df.to_csv("technical_indicators.csv")
trade_df.to_csv("trade_log_with_reasoning.csv")
df.to_csv("final_data_with_indicators.csv")

In [47]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score

# Step 1: Load your saved dataset containing technical indicators and signals
df = pd.read_csv("/home/justdial/Bob_Old_Data/Trading_bot/test_1lk_trade_msft/final_data_with_indicators.csv", 
                 index_col=0, parse_dates=True)
trade_log = pd.read_csv("/home/justdial/Bob_Old_Data/Trading_bot/test_1lk_trade_msft/trade_log_with_reasoning.csv", 
                        index_col=0, parse_dates=True)

# Step 2: Prepare features and target
feature_cols = [
    "MA44", "bb_bbm", "bb_bbh", "bb_bbl",
    "ichimoku_a", "ichimoku_b", "ichimoku_base", "ichimoku_conv",
    "volume", "volume_ma5", "volume_prediction"
]

# Drop rows with NaNs in features/signals
df_model = df.dropna(subset=feature_cols + ["signal"])

signal_map = {"buy": 1, "hold": 0, "sell": -1}
df_model.loc[:, "signal_label"] = df_model["signal"].map(signal_map)

X = df_model[feature_cols]
y = df_model["signal_label"]

# Step 3: Train/test split without shuffling (time series)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

# Step 4: Train Random Forest with balanced class weights
rf_clf = RandomForestClassifier(n_estimators=100, random_state=42, class_weight='balanced')
rf_clf.fit(X_train, y_train)

# Step 5: Predictions
y_pred = rf_clf.predict(X_test)

print("Classification Report (Random Forest):")
print(classification_report(y_test, y_pred, zero_division=0))
print("Accuracy:", accuracy_score(y_test, y_pred))

# Feature importance
importances = rf_clf.feature_importances_
feature_importance_df = pd.DataFrame({"feature": feature_cols, "importance": importances}).sort_values(by="importance", ascending=False)
print("\nFeature Importances:")
print(feature_importance_df)

# Step 6: Generate predicted signals for full dataset
df_model["predicted_signal"] = rf_clf.predict(X)
reverse_map = {v: k for k, v in signal_map.items()}
df_model["predicted_signal_label"] = df_model["predicted_signal"].map(reverse_map)

df_model.to_csv("/home/justdial/Bob_Old_Data/Trading_bot/test_1lk_trade_msft/model_predictions.csv")

print("\nSample predictions with features and signals:")
print(df_model[["signal", "predicted_signal_label"]].head())

# Step 7: Load or compute profit summaries to calculate total profit percentages
# You can load from your saved CSVs if available, otherwise compute from `trade_log`

try:
    daily_profit = pd.read_csv("/home/justdial/Bob_Old_Data/Trading_bot/test_1lk_trade_msft/daily_profit_summary.csv", 
                              index_col=0, parse_dates=True)
    weekly_profit = pd.read_csv("/home/justdial/Bob_Old_Data/Trading_bot/test_1lk_trade_msft/weekly_profit_summary.csv", 
                               index_col=0, parse_dates=True)
    monthly_profit = pd.read_csv("/home/justdial/Bob_Old_Data/Trading_bot/test_1lk_trade_msft/monthly_profit_summary.csv", 
                                index_col=0, parse_dates=True)
except FileNotFoundError:
    print("Profit summary CSV files not found, attempting to compute from trade_log")

    # Define a helper function to compute and save profit summaries from trade_log
    def profit_summary(trades, freq, initial_capital=100000):
        sells = trades[trades["trade_type"] == "sell"]
        if sells.empty:
            print(f"No sell trades found for freq={freq}")
            return pd.DataFrame()
        grouped = sells.resample(freq).agg({"profit": "sum"})
        grouped["profit_pct"] = (grouped["profit"] / initial_capital) * 100
        grouped = grouped.rename(columns={"profit": "total_profit"})
        return grouped

    daily_profit = profit_summary(trade_log, "D")
    weekly_profit = profit_summary(trade_log, "W")
    monthly_profit = profit_summary(trade_log, "M")

    # # Save profit summaries for future use
    # daily_profit.to_csv("/home/justdial/Bob_Old_Data/Trading_bot/test_1lk_trade_msft/daily_profit_summary.csv")
    # weekly_profit.to_csv("/home/justdial/Bob_Old_Data/Trading_bot/test_1lk_trade_msft/weekly_profit_summary.csv")
    # monthly_profit.to_csv("/home/justdial/Bob_Old_Data/Trading_bot/test_1lk_trade_msft/monthly_profit_summary.csv")

# Step 8: Calculate total profit percentages (sum over all periods)
total_daily_profit_pct = daily_profit["profit_pct"].sum()
total_weekly_profit_pct = weekly_profit["profit_pct"].sum()
total_monthly_profit_pct = monthly_profit["profit_pct"].sum()

print(f"Total Daily Profit Percentage: {total_daily_profit_pct:.2f}%")
print(f"Total Weekly Profit Percentage: {total_weekly_profit_pct:.2f}%")
print(f"Total Monthly Profit Percentage: {total_monthly_profit_pct:.2f}%")


/tmp/ipykernel_127560/1217633000.py:24: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



Classification Report (Random Forest):
              precision    recall  f1-score   support

          -1       0.00      0.00      0.00        20
           0       0.66      0.86      0.74       111
           1       0.86      0.76      0.81       126

    accuracy                           0.74       257
   macro avg       0.50      0.54      0.52       257
weighted avg       0.70      0.74      0.72       257

Accuracy: 0.7431906614785992

Feature Importances:
              feature  importance
3              bb_bbl    0.146734
0                MA44    0.122955
5          ichimoku_b    0.120394
2              bb_bbh    0.104183
6       ichimoku_base    0.099746
1              bb_bbm    0.095790
4          ichimoku_a    0.075873
10  volume_prediction    0.065015
9          volume_ma5    0.062622
7       ichimoku_conv    0.062437
8              volume    0.044250

Sample predictions with features and signals:
                    signal predicted_signal_label
2025-06-30 16:00:00   se

/tmp/ipykernel_127560/1217633000.py:50: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/tmp/ipykernel_127560/1217633000.py:52: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [5]:
import pandas as pd
import numpy as np
from ta.trend import IchimokuIndicator
from ta.volatility import BollingerBands

# Step 1: Load the Excel file without headers to inspect poorly formatted data
file_path = '/home/justdial/Bob_Old_Data/Trading_bot/stock_data/Adani enterprise 30 min.xlsx'

# Load entire Excel sheet without header to inspect where real data starts
raw_df = pd.read_excel(file_path, header=None)

# Inspect the first 50 rows to locate the actual data header row
print(raw_df.head(50))

# Step 2: Identify the header row
# From your sample, the columns for data start with 'Local Date', so find the row index of that
header_row_idx = raw_df[raw_df[0] == 'Local Date'].index[0]

# Reload file again setting the header row properly to parse the data
df = pd.read_excel(file_path, header=header_row_idx)

# Step 3: Rename columns to standard expected names
df = df.rename(columns={
    'Local Date': 'date',
    'Local Time': 'time',
    'Open': 'open',
    'High': 'high',
    'Low': 'low',
    'Close': 'close',
    'Volume': 'volume'
})

# Step 4: Combine date and time columns into a datetime index (if time column exists)
if 'time' in df.columns and not df['time'].isnull().all():
    df['datetime'] = pd.to_datetime(df['date'].astype(str) + ' ' + df['time'].astype(str), errors='coerce')
else:
    df['datetime'] = pd.to_datetime(df['date'].astype(str), errors='coerce')

# Drop rows where datetime could not be parsed (NaT)
df = df.dropna(subset=['datetime'])

df.set_index('datetime', inplace=True)

# Step 5: Keep necessary columns and convert to numeric (coerce errors to NaN)
expected_cols = ['open', 'high', 'low', 'close', 'volume']
df = df[expected_cols].apply(pd.to_numeric, errors='coerce')

df.dropna(inplace=True)  # Drop any rows with missing price data

# Step 6: Calculate Indicators

df["MA44"] = df["close"].rolling(window=44).mean()

bb = BollingerBands(close=df["close"], window=20, window_dev=2)
df["bb_bbm"] = bb.bollinger_mavg()
df["bb_bbh"] = bb.bollinger_hband()
df["bb_bbl"] = bb.bollinger_lband()

ich = IchimokuIndicator(high=df["high"], low=df["low"])
df["ichimoku_a"] = ich.ichimoku_a()
df["ichimoku_b"] = ich.ichimoku_b()
df["ichimoku_base"] = ich.ichimoku_base_line()
df["ichimoku_conv"] = ich.ichimoku_conversion_line()

# Drop rows with NaN indicator values
df.dropna(subset=["MA44", "bb_bbm", "bb_bbh", "bb_bbl", "ichimoku_a", "ichimoku_b", "ichimoku_base", "ichimoku_conv"], inplace=True)

# Step 7: Define signal logic functions

def get_signal(row):
    close = row["close"]
    ma = row["MA44"]
    ich_a = row["ichimoku_a"]
    ich_b = row["ichimoku_b"]
    bb_upper = row["bb_bbh"]
    bb_lower = row["bb_bbl"]

    if close > ma and close > ich_a and close > ich_b and close <= bb_lower * 1.01:
        return "buy"
    elif close < ma and close < ich_a and close < ich_b and close >= bb_upper * 0.99:
        return "sell"
    else:
        return "hold"

def signal_reason(row):
    close = row["close"]
    ma = row["MA44"]
    ich_a = row["ichimoku_a"]
    ich_b = row["ichimoku_b"]
    bb_upper = row["bb_bbh"]
    bb_lower = row["bb_bbl"]
    messages = []

    if close > ma:
        messages.append(f"Close ({close:.2f}) > MA44 ({ma:.2f})")
    else:
        messages.append(f"Close ({close:.2f}) <= MA44 ({ma:.2f})")

    if close > ich_a:
        messages.append(f"Close ({close:.2f}) > Ichimoku A ({ich_a:.2f})")
    else:
        messages.append(f"Close ({close:.2f}) <= Ichimoku A ({ich_a:.2f})")

    if close > ich_b:
        messages.append(f"Close ({close:.2f}) > Ichimoku B ({ich_b:.2f})")
    else:
        messages.append(f"Close ({close:.2f}) <= Ichimoku B ({ich_b:.2f})")

    buy_band = close <= bb_lower * 1.01
    sell_band = close >= bb_upper * 0.99

    if buy_band:
        messages.append(f"Close ({close:.2f}) <= Lower BB * 1.01 ({bb_lower * 1.01:.2f})")
    if sell_band:
        messages.append(f"Close ({close:.2f}) >= Upper BB * 0.99 ({bb_upper * 0.99:.2f})")

    return "; ".join(messages)

df["signal"] = df.apply(get_signal, axis=1)

# Step 8: Backtesting with initial capital ₹100,000

initial_cash = 100000.0
cash = initial_cash
position = 0.0
entry_price = 0.0
log = []

for idx, row in df.iterrows():
    signal = row["signal"]
    price = row["close"]
    reasoning = signal_reason(row)

    if signal == "buy" and position == 0:
        position = cash / price
        cash = 0
        entry_price = price
        log.append({
            "date": idx,
            "trade_type": "buy",
            "price": price,
            "units": position,
            "cash": cash,
            "signal": signal,
            "reason": reasoning,
            "position_type": "long"
        })

    elif signal == "sell" and position > 0:
        cash = position * price
        profit = cash - initial_cash
        profit_pct = (profit / initial_cash) * 100
        log.append({
            "date": idx,
            "trade_type": "sell",
            "price": price,
            "units": -position,
            "cash": cash,
            "profit": profit,
            "profit_pct": profit_pct,
            "signal": signal,
            "reason": reasoning,
            "position_type": "long"
        })
        position = 0
        entry_price = 0

# Final sell if position still open at last row
if position > 0:
    price = df["close"].iloc[-1]
    cash = price * position
    profit = cash - initial_cash
    profit_pct = (profit / initial_cash) * 100
    log.append({
        "date": df.index[-1],
        "trade_type": "sell",
        "price": price,
        "units": -position,
        "cash": cash,
        "profit": profit,
        "profit_pct": profit_pct,
        "signal": df["signal"].iloc[-1],
        "reason": signal_reason(df.iloc[-1]),
        "position_type": "long"
    })
    position = 0

trade_df = pd.DataFrame(log).set_index("date")

# Step 9: Profit summary

def profit_summary(trades, freq='M'):
    sells = trades[trades["trade_type"] == "sell"]
    if sells.empty:
        return pd.DataFrame()
    grouped = sells.resample(freq).agg({"profit": "sum"})
    grouped["profit_pct"] = (grouped["profit"] / initial_cash) * 100
    return grouped.rename(columns={"profit": "total_profit"})

daily_profit = profit_summary(trade_df, "D")
weekly_profit = profit_summary(trade_df, "W")
monthly_profit = profit_summary(trade_df, "M")

# Print summaries and last recommendation
print("===== Trade Log =====")
print(trade_df[[
    "trade_type", "price", "units", "cash", "profit", "profit_pct", "signal", "reason"
]])
print("\n===== Monthly Profit Summary =====")
print(monthly_profit)
print(f"Total Profit: ₹{monthly_profit['total_profit'].sum():,.2f}")
print(f"Total Profit Percentage: {(monthly_profit['profit_pct'].sum() if not monthly_profit.empty else 0):.2f}%")

last_signal = df["signal"].iloc[-1]
if last_signal == "buy":
    print("\nFinal Recommendation: GO LONG (BUY)")
elif last_signal == "sell":
    print("\nFinal Recommendation: SELL or STAY OUT")
else:
    print("\nFinal Recommendation: HOLD")


                                                   0                    1   \
0   Adani Enterprises Ltd | Price History         ...                  NaN   
1                                                 NaN                  NaN   
2                                                 NaN                  NaN   
3                                             ADEL.NS                  NaN   
4                                Interval: 30 Minutes                  NaN   
5   History Period: 04-Aug-2024 13:47 - 04-Aug-202...                  NaN   
6                                                 NaN                  NaN   
7                                                 NaN                  NaN   
8                                          VAP: Total            411714966   
9                                               Price               Volume   
10                                3,200.00 - 3,400.00               788050   
11                                3,000.00 - 3,200.00           

/tmp/ipykernel_142002/961880704.py:35: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['datetime'] = pd.to_datetime(df['date'].astype(str) + ' ' + df['time'].astype(str), errors='coerce')


KeyError: "None of ['date'] are in the columns"

In [19]:
import pandas as pd

file_path = '/home/justdial/Bob_Old_Data/Trading_bot/stock_data/Adani enterprise 30 min.xlsx'

raw_df = pd.read_excel(file_path, header=None)
print(raw_df.head(50))


                                                   0                    1   \
0   Adani Enterprises Ltd | Price History         ...                  NaN   
1                                                 NaN                  NaN   
2                                                 NaN                  NaN   
3                                             ADEL.NS                  NaN   
4                                Interval: 30 Minutes                  NaN   
5   History Period: 04-Aug-2024 13:47 - 04-Aug-202...                  NaN   
6                                                 NaN                  NaN   
7                                                 NaN                  NaN   
8                                          VAP: Total            411714966   
9                                               Price               Volume   
10                                3,200.00 - 3,400.00               788050   
11                                3,000.00 - 3,200.00           

In [20]:
header_row = raw_df[raw_df[0] == "Local Date"].index[0]

df = pd.read_excel(file_path, header=header_row)


In [21]:
df

,Local Date,Local Time,Close,Net,%Chg,Open,Low,High,Volume,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12
0,2025-08-04 13:00:00,2025-08-04 13:00:00,2363.60,0.00,0.000000,2363.60,2363.6,2363.60,NaN,NaN,NaN,NaN,NaN
1,2025-08-04 11:30:00,2025-08-04 11:30:00,2363.60,-1.10,-0.000465,2364.70,2363.6,2364.70,77.0,NaN,NaN,NaN,NaN
2,2025-08-04 11:00:00,2025-08-04 11:00:00,2364.70,0.70,0.000296,2363.80,2359.7,2367.90,104629.0,NaN,NaN,NaN,NaN
3,2025-08-04 10:30:00,2025-08-04 10:30:00,2364.00,1.60,0.000677,2362.00,2358.4,2368.70,42353.0,NaN,NaN,NaN,NaN
4,2025-08-04 10:00:00,2025-08-04 10:00:00,2362.40,-9.00,-0.003795,2371.30,2362.2,2374.70,38661.0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3988,2024-08-05 06:30:00,2024-08-05 06:30:00,3055.55,-45.55,-0.014688,3101.10,3053.0,3102.75,168656.0,NaN,NaN,NaN,NaN
3989,2024-08-05 06:00:00,2024-08-05 06:00:00,3101.10,-23.30,-0.007457,3124.55,3097.3,3124.55,133664.0,NaN,NaN,NaN,NaN
3990,2024-08-05 05:30:00,2024-08-05 05:30:00,3124.40,50.10,0.016296,3075.20,3065.9,3125.35,301560.0,NaN,NaN,NaN,NaN
3991,2024-08-05 05:00:00,2024-08-05 05:00:00,3074.30,NaN,NaN,3085.00,3062.0,3128.70,313461.0,NaN,NaN,NaN,NaN


In [22]:
df

,Local Date,Local Time,Close,Net,%Chg,Open,Low,High,Volume,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12
0,2025-08-04 13:00:00,2025-08-04 13:00:00,2363.60,0.00,0.000000,2363.60,2363.6,2363.60,NaN,NaN,NaN,NaN,NaN
1,2025-08-04 11:30:00,2025-08-04 11:30:00,2363.60,-1.10,-0.000465,2364.70,2363.6,2364.70,77.0,NaN,NaN,NaN,NaN
2,2025-08-04 11:00:00,2025-08-04 11:00:00,2364.70,0.70,0.000296,2363.80,2359.7,2367.90,104629.0,NaN,NaN,NaN,NaN
3,2025-08-04 10:30:00,2025-08-04 10:30:00,2364.00,1.60,0.000677,2362.00,2358.4,2368.70,42353.0,NaN,NaN,NaN,NaN
4,2025-08-04 10:00:00,2025-08-04 10:00:00,2362.40,-9.00,-0.003795,2371.30,2362.2,2374.70,38661.0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3988,2024-08-05 06:30:00,2024-08-05 06:30:00,3055.55,-45.55,-0.014688,3101.10,3053.0,3102.75,168656.0,NaN,NaN,NaN,NaN
3989,2024-08-05 06:00:00,2024-08-05 06:00:00,3101.10,-23.30,-0.007457,3124.55,3097.3,3124.55,133664.0,NaN,NaN,NaN,NaN
3990,2024-08-05 05:30:00,2024-08-05 05:30:00,3124.40,50.10,0.016296,3075.20,3065.9,3125.35,301560.0,NaN,NaN,NaN,NaN
3991,2024-08-05 05:00:00,2024-08-05 05:00:00,3074.30,NaN,NaN,3085.00,3062.0,3128.70,313461.0,NaN,NaN,NaN,NaN


In [23]:
# Strip duplicates from 'Local Date' if any
df['Local Date'] = df['Local Date'].astype(str).apply(lambda x: x.split()[0] + ' ' + x.split()[1] if len(x.split()) >= 2 else x)

# Similarly for 'Local Time', if exists and malformed similarly
df['Local Time'] = df['Local Time'].astype(str).apply(lambda x: x.split()[0] if len(x.split()) >= 1 else x)

df = df.rename(columns={
    'Local Date': 'date',
    'Local Time': 'time',
    'Open': 'open',
    'High': 'high',
    'Low': 'low',
    'Close': 'close',
    'Volume': 'volume'
})  

# Combine date and time into a datetime index
if 'time' in df.columns and not df['time'].isnull().all():
    df['datetime'] = pd.to_datetime(df['date'].astype(str) + ' ' + df['time'].astype(str), errors='coerce')
else:
    df['datetime'] = pd.to_datetime(df['date'].astype(str), errors='coerce')
df.set_index('datetime', inplace=True)

# Keep only necessary columns and convert to numeric
expected_cols = ['open', 'high', 'low', 'close', 'volume']
df = df[expected_cols].apply(pd.to_numeric, errors='coerce')
df.dropna(inplace=True)  # Drop any rows with missing price data

# Calculate Indicators
df["MA44"] = df["close"].rolling(window=44).mean()
bb = BollingerBands(close=df["close"], window=20, window_dev=2)
df["bb_bbm"] = bb.bollinger_mavg()
df["bb_bbh"] = bb.bollinger_hband()
df["bb_bbl"] = bb.bollinger_lband()
ich = IchimokuIndicator(high=df["high"], low=df["low"])
df["ichimoku_a"] = ich.ichimoku_a()
df["ichimoku_b"] = ich.ichimoku_b()
df["ichimoku_base"] = ich.ichimoku_base_line()
df["ichimoku_conv"] = ich.ichimoku_conversion_line()
df.dropna(subset=["MA44", "bb_bbm", "bb_bbh", "bb_bbl", "ichimoku_a", "ichimoku_b", "ichimoku_base", "ichimoku_conv"], inplace=True)
# Define signal logic functions

def get_signal(row):
    close = row["close"]
    ma = row["MA44"]
    ich_a = row["ichimoku_a"]
    ich_b = row["ichimoku_b"]
    bb_upper = row["bb_bbh"]
    bb_lower = row["bb_bbl"]

    if close > ma and close > ich_a and close > ich_b and close <= bb_lower * 1.01:
        return "buy"
    elif close < ma and close < ich_a and close < ich_b and close >= bb_upper * 0.99:
        return "sell"
    else:
        return "hold"

def signal_reason(row):
    close = row["close"]
    ma = row["MA44"]
    ich_a = row["ichimoku_a"]
    ich_b = row["ichimoku_b"]
    bb_upper = row["bb_bbh"]
    bb_lower = row["bb_bbl"]
    messages = []

    if close > ma:
        messages.append(f"Close ({close:.2f}) > MA44 ({ma:.2f})")
    else:
        messages.append(f"Close ({close:.2f}) <= MA44 ({ma:.2f})")

    if close > ich_a:
        messages.append(f"Close ({close:.2f}) > Ichimoku A ({ich_a:.2f})")
    else:
        messages.append(f"Close ({close:.2f}) <= Ichimoku A ({ich_a:.2f})")

    if close > ich_b:
        messages.append(f"Close ({close:.2f}) > Ichimoku B ({ich_b:.2f})")
    else:
        messages.append(f"Close ({close:.2f}) <= Ichimoku B ({ich_b:.2f})")

    buy_band = close <= bb_lower * 1.01
    sell_band = close >= bb_upper * 0.99

    if buy_band:
        messages.append(f"Close ({close:.2f}) <= Lower BB * 1.01 ({bb_lower * 1.01:.2f})")
    if sell_band:
        messages.append(f"Close ({close:.2f}) >= Upper BB * 0.99 ({bb_upper * 0.99:.2f})")

    return "; ".join(messages)

df["signal"] = df.apply(get_signal, axis=1)
df["signal_reason"] = df.apply(signal_reason, axis=1)

# Backtesting with initial capital ₹100,000
initial_cash = 100000.0
cash = initial_cash

position = 0.0
entry_price = 0.0
log = []

for idx, row in df.iterrows():
    signal = row["signal"]
    if signal == "buy" and cash > 0:
        position = cash / row["close"]
        entry_price = row["close"]
        cash = 0
        log.append((idx, "buy", entry_price, position))
    elif signal == "sell" and position > 0:
        cash = position * row["close"]
        log.append((idx, "sell", row["close"], position))
        position = 0
        entry_price = 0.0
# Final sell if still holding
if position > 0:
    last_price = df["close"].iloc[-1]
    cash = position * last_price
    log.append((df.index[-1], "sell", last_price, position))
    position = 0.0
trade_df = pd.DataFrame(log, columns=["date", "trade_type", "price", "units"]).set_index("date")
# Profit summary function
def profit_summary(trades, freq='M'):
    sells = trades[trades["trade_type"] == "sell"]
    if sells.empty:
        return pd.DataFrame()
    grouped = sells.resample(freq).agg({"price": "sum"})
    grouped["profit"] = grouped["price"] - initial_cash
    grouped["profit_pct"] = (grouped["profit"] / initial_cash) * 100
    return grouped.rename(columns={"price": "total_profit"})

daily_profit = profit_summary(trade_df, "D")
weekly_profit = profit_summary(trade_df, "W")
monthly_profit = profit_summary(trade_df, "M")

# Print summaries and last recommendation
print("===== Trade Log =====")
print(trade_df)
print("\n===== Monthly Profit Summary =====")
print(monthly_profit)
print(f"Total Profit: ₹{monthly_profit['total_profit'].sum():,.2f}")
print(f"Total Profit Percentage: {(monthly_profit['profit_pct'].sum() if not monthly_profit.empty else 0):.2f}%")

last_signal = df["signal"].iloc[-1]
if last_signal == "buy":
    print("\nFinal Recommendation: GO LONG (BUY)")
elif last_signal == "sell":
    print("\nFinal Recommendation: SELL or STAY OUT")
else:
    print("\nFinal Recommendation: HOLD")
# import pandas as pd
# import numpy as np
# from ta.trend import IchimokuIndicator
# from ta.volatility import BollingerBands
# # Step 1: Load the Excel file without headers to inspect poorly formatted data
# file_path = '/home/justdial/Bob_Old_Data/Trading_bot/stock_data/Adani enterprise 30 min.xlsx'
# raw_df = pd.read_excel(file_path, header=None)
# # Inspect the first 50 rows to locate the actual data header row
# print(raw_df.head(50))  
# # Step 2: Identify the header row
# # From your sample, the columns for data start with 'Local Date', so find the row index of that
# header_row_idx = raw_df[raw_df[0] == 'Local Date'].index[0]
# # Reload file again setting the header row properly to parse the data
# df = pd.read_excel(file_path, header=header_row_idx)
# # Step 3: Rename columns to standard expected names
# df = df.rename(columns={
#     'Local Date': 'date',
#     'Local Time': 'time',
#     'Open': 'open',
#     'High': 'high',
#     'Low': 'low',
#     'Close': 'close',
#     'Volume': 'volume'
# })
# # Step 4: Combine date and time columns into a datetime index (if time column exists)
# if 'time' in df.columns and not df['time'].isnull().all():
#     df['datetime'] = pd.to_datetime(df['date'].astype(str) + ' ' + df['time'].astype(str), errors='coerce')
# else:
#     df['datetime'] = pd.to_datetime(df['date'].astype(str), errors='coerce')
# df.set_index('datetime', inplace=True)
# # Step 5: Keep necessary columns and convert to numeric (coerce errors to NaN)
# expected_cols = ['open', 'high', 'low', 'close', 'volume']
# df = df[expected_cols].apply(pd.to_numeric, errors='coerce')
# df.dropna(inplace=True)  # Drop any rows with missing price data


/tmp/ipykernel_142002/4238213525.py:19: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['datetime'] = pd.to_datetime(df['date'].astype(str) + ' ' + df['time'].astype(str), errors='coerce')


AttributeError: 'NaTType' object has no attribute 'normalize'

In [39]:
import pandas as pd
from ta.trend import IchimokuIndicator
from ta.volatility import BollingerBands

# Step 1: Read the Excel file and find header row
file_path = '/home/justdial/Bob_Old_Data/Trading_bot/stock_data/Adani enterprise 30 min.xlsx'
raw_df = pd.read_excel(file_path, header=None)
header_row = raw_df[raw_df[0] == 'Local Date'].index[0]

# Step 2: Load with proper header
df = pd.read_excel(file_path, header=header_row)

# Step 3: Parse the 'Local Date' column as datetime and set as index
df['datetime'] = pd.to_datetime(df['Local Date'], errors='coerce')

# Drop rows where date parsing failed
df = df.dropna(subset=['datetime'])

# Set datetime as index
df.set_index('datetime', inplace=True)

# Step 4: Rename relevant columns to standard lowercase names
df.rename(columns={
    'Open': 'open',
    'High': 'high',
    'Low': 'low',
    'Close': 'close',
    'Volume': 'volume'
}, inplace=True)

# Step 5: Convert numeric columns properly and drop any rows with missing data
for col in ['open', 'high', 'low', 'close', 'volume']:
    df[col] = pd.to_numeric(df[col], errors='coerce')
df = df.dropna(subset=['open', 'high', 'low', 'close', 'volume'])


# Keep only necessary columns and convert to numeric
expected_cols = ['open', 'high', 'low', 'close', 'volume']
df = df[expected_cols].apply(pd.to_numeric, errors='coerce')
df.dropna(inplace=True)  # Drop any rows with missing price data


# Step 6: Calculate indicators as normal
df['MA44'] = df['close'].rolling(window=44).mean()

bb = BollingerBands(close=df['close'], window=20, window_dev=2)
df['bb_bbm'] = bb.bollinger_mavg()
df['bb_bbh'] = bb.bollinger_hband()
df['bb_bbl'] = bb.bollinger_lband()

ich = IchimokuIndicator(high=df['high'], low=df['low'])
df['ichimoku_a'] = ich.ichimoku_a()
df['ichimoku_b'] = ich.ichimoku_b()
df['ichimoku_base'] = ich.ichimoku_base_line()
df['ichimoku_conv'] = ich.ichimoku_conversion_line()

# Drop rows where indicators cannot be calculated
df.dropna(subset=['MA44', 'bb_bbm', 'bb_bbh', 'bb_bbl',
                  'ichimoku_a', 'ichimoku_b', 'ichimoku_base', 'ichimoku_conv'], inplace=True)

print(df.head())


                       open    high     low   close   volume         MA44  \
datetime                                                                    
2025-07-30 11:00:00  2538.9  2539.6  2525.2  2527.0  55234.0  2417.763636   
2025-07-30 10:30:00  2541.9  2543.8  2538.2  2538.2   9774.0  2421.731818   
2025-07-30 10:00:00  2541.5  2543.6  2537.8  2541.9   8110.0  2425.759091   
2025-07-30 09:30:00  2539.6  2541.9  2536.0  2541.5  13013.0  2429.793182   
2025-07-30 09:00:00  2544.9  2546.0  2537.6  2539.6   8662.0  2433.820455   

                       bb_bbm       bb_bbh       bb_bbl  ichimoku_a  \
datetime                                                              
2025-07-30 11:00:00  2473.530  2545.594280  2401.465720    2484.550   
2025-07-30 10:30:00  2479.410  2552.424297  2406.395703    2486.650   
2025-07-30 10:00:00  2485.335  2558.439317  2412.230683    2487.900   
2025-07-30 09:30:00  2491.290  2562.290927  2420.289073    2491.400   
2025-07-30 09:00:00  2496.875  256

In [46]:
# find unique datetime 
unique_dates = df.index.unique()
# print(f"Unique Dates Count: nique_dates)}")

In [50]:
unique_dates

DatetimeIndex(['2025-07-30 11:00:00', '2025-07-30 10:30:00',
               '2025-07-30 10:00:00', '2025-07-30 09:30:00',
               '2025-07-30 09:00:00', '2025-07-30 08:30:00',
               '2025-07-30 08:00:00', '2025-07-30 07:30:00',
               '2025-07-30 07:00:00', '2025-07-30 06:30:00',
               ...
               '2024-08-05 09:30:00', '2024-08-05 09:00:00',
               '2024-08-05 08:30:00', '2024-08-05 08:00:00',
               '2024-08-05 07:30:00', '2024-08-05 07:00:00',
               '2024-08-05 06:30:00', '2024-08-05 06:00:00',
               '2024-08-05 05:30:00', '2024-08-05 05:00:00'],
              dtype='datetime64[ns]', name='datetime', length=3447, freq=None)

In [41]:
import pandas as pd
from ta.trend import IchimokuIndicator
from ta.volatility import BollingerBands

# Path to your Excel file
file_path = '/home/justdial/Bob_Old_Data/Trading_bot/stock_data/Adani enterprise 30 min.xlsx'

# Step 1: Identify header row dynamically by looking for 'Local Date' in first column
raw_df = pd.read_excel(file_path, header=None)
header_row = raw_df[raw_df[0] == 'Local Date'].index[0]

# Step 2: Load Excel with proper header row
df = pd.read_excel(file_path, header=header_row)

# Step 3: Parse 'Local Date' column as datetime index (it has full datetime strings already)
df['datetime'] = pd.to_datetime(df['Local Date'], errors='coerce')

# Drop rows where datetime parsing failed
df = df.dropna(subset=['datetime'])
df = df.set_index('datetime')

# Step 4: Rename columns to expected lowercase standardized names
df.rename(columns={
    'Open': 'open',
    'High': 'high',
    'Low': 'low',
    'Close': 'close',
    'Volume': 'volume'
}, inplace=True)

# Step 5: Convert price and volume columns to numeric and drop rows with missing values
for col in ['open', 'high', 'low', 'close', 'volume']:
    df[col] = pd.to_numeric(df[col], errors='coerce')
df = df.dropna(subset=['open', 'high', 'low', 'close', 'volume'])

# Step 6: Calculate technical indicators
df['MA44'] = df['close'].rolling(window=44).mean()

bb = BollingerBands(close=df['close'], window=20, window_dev=2)
df['bb_bbm'] = bb.bollinger_mavg()
df['bb_bbh'] = bb.bollinger_hband()
df['bb_bbl'] = bb.bollinger_lband()

ich = IchimokuIndicator(high=df['high'], low=df['low'])
df['ichimoku_a'] = ich.ichimoku_a()
df['ichimoku_b'] = ich.ichimoku_b()
df['ichimoku_base'] = ich.ichimoku_base_line()
df['ichimoku_conv'] = ich.ichimoku_conversion_line()

# Drop rows where indicator data is missing
df.dropna(subset=['MA44', 'bb_bbm', 'bb_bbh', 'bb_bbl', 'ichimoku_a', 'ichimoku_b', 'ichimoku_base', 'ichimoku_conv'], inplace=True)

print("Prepared Data Sample:")
print(df.head(10))


Prepared Data Sample:
                             Local Date          Local Time   close   Net  \
datetime                                                                    
2025-07-30 11:00:00 2025-07-30 11:00:00 2025-07-30 11:00:00  2527.0 -11.2   
2025-07-30 10:30:00 2025-07-30 10:30:00 2025-07-30 10:30:00  2538.2  -3.7   
2025-07-30 10:00:00 2025-07-30 10:00:00 2025-07-30 10:00:00  2541.9   0.4   
2025-07-30 09:30:00 2025-07-30 09:30:00 2025-07-30 09:30:00  2541.5   1.9   
2025-07-30 09:00:00 2025-07-30 09:00:00 2025-07-30 09:00:00  2539.6  -5.2   
2025-07-30 08:30:00 2025-07-30 08:30:00 2025-07-30 08:30:00  2544.8   2.8   
2025-07-30 08:00:00 2025-07-30 08:00:00 2025-07-30 08:00:00  2542.0  -4.3   
2025-07-30 07:30:00 2025-07-30 07:30:00 2025-07-30 07:30:00  2546.3  -2.4   
2025-07-30 07:00:00 2025-07-30 07:00:00 2025-07-30 07:00:00  2548.7   6.3   
2025-07-30 06:30:00 2025-07-30 06:30:00 2025-07-30 06:30:00  2542.4   4.4   

                         %Chg    open     low    high

In [42]:
df

,Local Date,Local Time,close,Net,%Chg,open,low,high,volume,Unnamed: 9,...,Unnamed: 11,Unnamed: 12,MA44,bb_bbm,bb_bbh,bb_bbl,ichimoku_a,ichimoku_b,ichimoku_base,ichimoku_conv
datetime,,,,,,,,,,,,,,,,,,,,,
2025-07-30 11:00:00,2025-07-30 11:00:00,2025-07-30 11:00:00,2527.00,-11.20,-0.004413,2538.90,2525.2,2539.60,55234.0,NaN,...,NaN,NaN,2417.763636,2473.5300,2545.594280,2401.465720,2484.5500,2437.3,2461.300,2507.800
2025-07-30 10:30:00,2025-07-30 10:30:00,2025-07-30 10:30:00,2538.20,-3.70,-0.001456,2541.90,2538.2,2543.80,9774.0,NaN,...,NaN,NaN,2421.731818,2479.4100,2552.424297,2406.395703,2486.6500,2439.4,2463.400,2509.900
2025-07-30 10:00:00,2025-07-30 10:00:00,2025-07-30 10:00:00,2541.90,0.40,0.000157,2541.50,2537.8,2543.60,8110.0,NaN,...,NaN,NaN,2425.759091,2485.3350,2558.439317,2412.230683,2487.9000,2439.4,2465.900,2509.900
2025-07-30 09:30:00,2025-07-30 09:30:00,2025-07-30 09:30:00,2541.50,1.90,0.000748,2539.60,2536.0,2541.90,13013.0,NaN,...,NaN,NaN,2429.793182,2491.2900,2562.290927,2420.289073,2491.4000,2439.4,2472.900,2509.900
2025-07-30 09:00:00,2025-07-30 09:00:00,2025-07-30 09:00:00,2539.60,-5.20,-0.002043,2544.90,2537.6,2546.00,8662.0,NaN,...,NaN,NaN,2433.820455,2496.8750,2564.546822,2429.203178,2493.5750,2440.5,2475.650,2511.500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-08-05 07:00:00,2024-08-05 07:00:00,2024-08-05 07:00:00,3047.40,-8.15,-0.002667,3055.00,3017.7,3065.00,249294.0,NaN,...,NaN,NaN,3120.231818,3079.2325,3164.063363,2994.401637,3051.7375,3119.5,3072.475,3031.000
2024-08-05 06:30:00,2024-08-05 06:30:00,2024-08-05 06:30:00,3055.55,-45.55,-0.014688,3101.10,3053.0,3102.75,168656.0,NaN,...,NaN,NaN,3116.903409,3076.1100,3159.589720,2992.630280,3061.1750,3119.5,3072.475,3049.875
2024-08-05 06:00:00,2024-08-05 06:00:00,2024-08-05 06:00:00,3101.10,-23.30,-0.007457,3124.55,3097.3,3124.55,133664.0,NaN,...,NaN,NaN,3114.673864,3074.8150,3155.864967,2993.765033,3065.3375,3119.5,3069.900,3060.775


In [52]:
import pandas as pd
from ta.trend import IchimokuIndicator
from ta.volatility import BollingerBands

# Step 1: Load the Excel file and identify header row containing 'Local Date'
file_path = '/home/justdial/Bob_Old_Data/Trading_bot/stock_data/Adani enterprise 30 min.xlsx'
raw_df = pd.read_excel(file_path, header=None)
header_row = raw_df[raw_df[0] == 'Local Date'].index[0]

# Step 2: Load data using the identified header row
df = pd.read_excel(file_path, header=header_row)

# Step 3: Parse 'Local Date' column as datetime index (do NOT concatenate with 'Local Time')
df['datetime'] = pd.to_datetime(df['Local Date'], errors='coerce')
df = df.dropna(subset=['datetime'])
df.set_index('datetime', inplace=True)

# Step 4: Rename columns to standard lowercase names
df.rename(columns={
    'Open': 'open',
    'High': 'high',
    'Low': 'low',
    'Close': 'close',
    'Volume': 'volume'
}, inplace=True)

# Step 5: Convert price and volume columns to numeric and drop rows with any missing values
for col in ['open', 'high', 'low', 'close', 'volume']:
    df[col] = pd.to_numeric(df[col], errors='coerce')
df.dropna(subset=['open', 'high', 'low', 'close', 'volume'], inplace=True)

# Step 6: Calculate technical indicators
df['MA44'] = df['close'].rolling(window=44).mean()

bb = BollingerBands(close=df['close'], window=20, window_dev=2)
df['bb_bbm'] = bb.bollinger_mavg()
df['bb_bbh'] = bb.bollinger_hband()
df['bb_bbl'] = bb.bollinger_lband()

ich = IchimokuIndicator(high=df['high'], low=df['low'])
df['ichimoku_a'] = ich.ichimoku_a()
df['ichimoku_b'] = ich.ichimoku_b()
df['ichimoku_base'] = ich.ichimoku_base_line()
df['ichimoku_conv'] = ich.ichimoku_conversion_line()

# Drop rows where indicator calculations resulted in NaNs
df.dropna(subset=['MA44', 'bb_bbm', 'bb_bbh', 'bb_bbl', 
                  'ichimoku_a', 'ichimoku_b', 'ichimoku_base', 'ichimoku_conv'], inplace=True)

# Now the DataFrame is clean and ready for further analysis and backtesting
print(df.head(10))


                             Local Date          Local Time   close   Net  \
datetime                                                                    
2025-07-30 11:00:00 2025-07-30 11:00:00 2025-07-30 11:00:00  2527.0 -11.2   
2025-07-30 10:30:00 2025-07-30 10:30:00 2025-07-30 10:30:00  2538.2  -3.7   
2025-07-30 10:00:00 2025-07-30 10:00:00 2025-07-30 10:00:00  2541.9   0.4   
2025-07-30 09:30:00 2025-07-30 09:30:00 2025-07-30 09:30:00  2541.5   1.9   
2025-07-30 09:00:00 2025-07-30 09:00:00 2025-07-30 09:00:00  2539.6  -5.2   
2025-07-30 08:30:00 2025-07-30 08:30:00 2025-07-30 08:30:00  2544.8   2.8   
2025-07-30 08:00:00 2025-07-30 08:00:00 2025-07-30 08:00:00  2542.0  -4.3   
2025-07-30 07:30:00 2025-07-30 07:30:00 2025-07-30 07:30:00  2546.3  -2.4   
2025-07-30 07:00:00 2025-07-30 07:00:00 2025-07-30 07:00:00  2548.7   6.3   
2025-07-30 06:30:00 2025-07-30 06:30:00 2025-07-30 06:30:00  2542.4   4.4   

                         %Chg    open     low    high   volume  Unnamed: 9 

In [59]:
import pandas as pd
from ta.trend import IchimokuIndicator
from ta.volatility import BollingerBands

# File path
file_path = '/home/justdial/Bob_Old_Data/Trading_bot/stock_data/Adani enterprise 30 min.xlsx'

# Load raw Excel without header to find actual header row
raw_df = pd.read_excel(file_path, header=None)

# Locate header row (row containing 'Local Date')
header_row = raw_df[raw_df[0] == 'Local Date'].index[0]

# Read Excel with proper header row
df = pd.read_excel(file_path, header=header_row)

# Parse 'Local Date' to datetime, it's full timestamp already
df['datetime'] = pd.to_datetime(df['Local Date'], errors='coerce')

# Drop rows where datetime conversion failed
df.dropna(subset=['datetime'], inplace=True)

# Set datetime as index
df.set_index('datetime', inplace=True)

# Rename cols, convert to numeric, drop incomplete rows
df.rename(columns={
    'Open': 'open', 'High': 'high', 'Low': 'low',
    'Close': 'close', 'Volume': 'volume'
}, inplace=True)

for col in ['open', 'high', 'low', 'close', 'volume']:
    df[col] = pd.to_numeric(df[col], errors='coerce')

df.dropna(subset=['open', 'high', 'low', 'close', 'volume'], inplace=True)

# Sort by datetime ascending (older first)
df.sort_index(inplace=True)

# Optional: Inspect all unique dates to confirm data exists for missing days
dates = df.index.normalize().unique()
print("Available dates:")
print(dates)

# Check if the days you mention are in available dates:
missing_dates = ['2025-07-31', '2025-08-01', '2025-08-04']
for d in missing_dates:
    if pd.Timestamp(d) in dates:
        print(f"{d} found in data")
    else:
        print(f"{d} NOT found in data")

# Keep only necessary columns and convert to numeric
expected_cols = ['open', 'high', 'low', 'close', 'volume']
df = df[expected_cols].apply(pd.to_numeric, errors='coerce')
df.dropna(inplace=True)  # Drop any rows with missing price data

# Calculate indicators (optional, as before)
df['MA44'] = df['close'].rolling(window=44).mean()
bb = BollingerBands(close=df['close'], window=20, window_dev=2)
df['bb_bbm'] = bb.bollinger_mavg()
df['bb_bbh'] = bb.bollinger_hband()
df['bb_bbl'] = bb.bollinger_lband()
ich = IchimokuIndicator(high=df['high'], low=df['low'])
df['ichimoku_a'] = ich.ichimoku_a()
df['ichimoku_b'] = ich.ichimoku_b()
df['ichimoku_base'] = ich.ichimoku_base_line()
df['ichimoku_conv'] = ich.ichimoku_conversion_line()
df.dropna(subset=['MA44', 'bb_bbm', 'bb_bbh', 'bb_bbl', 
                  'ichimoku_a', 'ichimoku_b', 'ichimoku_base', 'ichimoku_conv'], inplace=True)

print(df.head())


Available dates:
DatetimeIndex(['2024-08-05', '2024-08-06', '2024-08-07', '2024-08-08',
               '2024-08-09', '2024-08-12', '2024-08-13', '2024-08-14',
               '2024-08-16', '2024-08-19',
               ...
               '2025-07-22', '2025-07-23', '2025-07-24', '2025-07-25',
               '2025-07-28', '2025-07-29', '2025-07-30', '2025-07-31',
               '2025-08-01', '2025-08-04'],
              dtype='datetime64[ns]', name='datetime', length=250, freq=None)
2025-07-31 found in data
2025-08-01 found in data
2025-08-04 found in data
                        open    high      low    close    volume         MA44  \
datetime                                                                        
2024-08-08 05:30:00  3185.00  3190.3  3168.00  3169.65  175996.0  3110.137500   
2024-08-08 06:00:00  3169.60  3194.6  3162.10  3192.30  259477.0  3112.819318   
2024-08-08 06:30:00  3192.00  3226.4  3186.15  3206.00  286073.0  3114.673864   
2024-08-08 07:00:00  3206.65  3219.

In [62]:
df.head(30)

,open,high,low,close,volume,MA44,bb_bbm,bb_bbh,bb_bbl,ichimoku_a,ichimoku_b,ichimoku_base,ichimoku_conv
datetime,,,,,,,,,,,,,
2024-08-08 05:30:00,3185.00,3190.30,3168.00,3169.65,175996.0,3110.137500,3145.3025,3218.758327,3071.846673,3153.1375,3100.275,3130.775,3175.500
2024-08-08 06:00:00,3169.60,3194.60,3162.10,3192.30,259477.0,3112.819318,3150.4725,3221.847198,3079.097802,3153.4375,3100.275,3130.775,3176.100
2024-08-08 06:30:00,3192.00,3226.40,3186.15,3206.00,286073.0,3114.673864,3156.6200,3224.815912,3088.424088,3164.8625,3111.700,3142.200,3187.525
2024-08-08 07:00:00,3206.65,3219.80,3196.80,3199.20,91860.0,3116.903409,3162.6675,3222.988585,3102.346415,3166.4500,3111.700,3142.200,3190.700
2024-08-08 07:30:00,3198.10,3212.80,3193.65,3202.00,65450.0,3120.231818,3169.1325,3215.630105,3122.634895,3168.2250,3111.700,3142.200,3194.250
2024-08-08 08:00:00,3202.00,3224.00,3194.80,3221.15,86006.0,3124.180682,3174.0900,3220.572143,3127.607857,3168.2250,3111.700,3142.200,3194.250
2024-08-08 08:30:00,3221.20,3242.00,3214.45,3216.30,244024.0,3128.371591,3178.4575,3223.547435,3133.367565,3176.0250,3119.500,3150.000,3202.050
2024-08-08 09:00:00,3215.00,3217.00,3178.60,3181.05,129160.0,3131.527273,3179.2100,3223.944144,3134.475856,3176.0250,3119.500,3150.000,3202.050
2024-08-08 09:30:00,3182.75,3206.10,3178.00,3196.00,125649.0,3135.888636,3181.4575,3224.801730,3138.113270,3176.0250,3119.500,3150.000,3202.050


In [ ]:
import pandas as pd
from ta.trend import IchimokuIndicator
from ta.volatility import BollingerBands

# ===== Step 1: Load and Prepare Data =====

# File path
file_path = '/home/justdial/Bob_Old_Data/Trading_bot/stock_data/Adani enterprise 30 min.xlsx'

# Load raw Excel without header to find actual header row
raw_df = pd.read_excel(file_path, header=None)
header_row = raw_df[raw_df[0] == 'Local Date'].index[0]
df = pd.read_excel(file_path, header=header_row)

# Parse datetime and set as index
df['datetime'] = pd.to_datetime(df['Local Date'], errors='coerce')
df.dropna(subset=['datetime'], inplace=True)
df.set_index('datetime', inplace=True)

# Rename and convert columns
df.rename(columns={
    'Open': 'open', 'High': 'high', 'Low': 'low',
    'Close': 'close', 'Volume': 'volume'
}, inplace=True)

for col in ['open', 'high', 'low', 'close', 'volume']:
    df[col] = pd.to_numeric(df[col], errors='coerce')

df.dropna(subset=['open', 'high', 'low', 'close', 'volume'], inplace=True)
df.sort_index(inplace=True)

# Check available dates
dates = df.index.normalize().unique()
print("Available dates:")
print(dates)

missing_dates = ['2025-07-31', '2025-08-01', '2025-08-04']
for d in missing_dates:
    if pd.Timestamp(d) in dates:
        print(f"{d} found in data")
    else:
        print(f"{d} NOT found in data")

# Keep only necessary columns and convert to numeric
expected_cols = ['open', 'high', 'low', 'close', 'volume']
df = df[expected_cols].apply(pd.to_numeric, errors='coerce')
df.dropna(inplace=True)  # Drop any rows with missing price data


# ===== Step 2: Technical Indicators =====

df['MA44'] = df['close'].rolling(window=44).mean()

bb = BollingerBands(close=df['close'], window=20, window_dev=2)
df['bb_bbm'] = bb.bollinger_mavg()
df['bb_bbh'] = bb.bollinger_hband()
df['bb_bbl'] = bb.bollinger_lband()

ich = IchimokuIndicator(high=df['high'], low=df['low'])
df['ichimoku_a'] = ich.ichimoku_a()
df['ichimoku_b'] = ich.ichimoku_b()
df['ichimoku_base'] = ich.ichimoku_base_line()
df['ichimoku_conv'] = ich.ichimoku_conversion_line()

df.dropna(subset=[
    'MA44', 'bb_bbm', 'bb_bbh', 'bb_bbl', 
    'ichimoku_a', 'ichimoku_b', 'ichimoku_base', 'ichimoku_conv'
], inplace=True)

# ===== Step 3: Stop Loss & Target Strategy =====

# Entry price = Close
df['entry_price'] = df['close']

# Define risk and reward percentages (2% risk, 4% reward)
risk_pct = 0.02
reward_pct = 0.04

# Compute Stop Loss and Target Price
df['stop_loss'] = df['entry_price'] * (1 - risk_pct)
df['target'] = df['entry_price'] * (1 + reward_pct)

# ===== Step 4: Trade Outcome Simulation =====

def simulate_trade_outcome(df, lookahead=5):
    outcomes = []
    for i in range(len(df) - lookahead):
        entry = df['entry_price'].iloc[i]
        stop_loss = df['stop_loss'].iloc[i]
        target = df['target'].iloc[i]
        lows = df['low'].iloc[i+1:i+1+lookahead]
        highs = df['high'].iloc[i+1:i+1+lookahead]

        sl_hit = lows.min() <= stop_loss
        tp_hit = highs.max() >= target

        if sl_hit and tp_hit:
            # Optional: could choose whichever comes first
            outcomes.append('Both Hit')
        elif sl_hit:
            outcomes.append('Stop Loss Hit')
        elif tp_hit:
            outcomes.append('Target Hit')
        else:
            outcomes.append('No Trigger')

    outcomes += [None] * lookahead  # Pad to match DataFrame length
    return outcomes

df['trade_outcome'] = simulate_trade_outcome(df)

# ===== Step 5: Output & Summary =====

print("\nTrade Outcome Counts:")
print(df['trade_outcome'].value_counts(dropna=True))

print("\nSample Data:")
print(df[['entry_price', 'stop_loss', 'target', 'trade_outcome']].tail(15))


Available dates:
DatetimeIndex(['2024-08-05', '2024-08-06', '2024-08-07', '2024-08-08',
               '2024-08-09', '2024-08-12', '2024-08-13', '2024-08-14',
               '2024-08-16', '2024-08-19',
               ...
               '2025-07-22', '2025-07-23', '2025-07-24', '2025-07-25',
               '2025-07-28', '2025-07-29', '2025-07-30', '2025-07-31',
               '2025-08-01', '2025-08-04'],
              dtype='datetime64[ns]', name='datetime', length=250, freq=None)
2025-07-31 found in data
2025-08-01 found in data
2025-08-04 found in data

Trade Outcome Counts:
trade_outcome
No Trigger       3114
Stop Loss Hit     255
Target Hit         71
Both Hit            2
Name: count, dtype: int64

Sample Data:
                     entry_price  stop_loss    target trade_outcome
datetime                                                           
2025-08-01 11:30:00       2350.9   2303.882  2444.936    No Trigger
2025-08-04 05:00:00       2357.9   2310.742  2452.216    No Trigger
202

In [66]:
df

,open,high,low,close,volume,MA44,bb_bbm,bb_bbh,bb_bbl,ichimoku_a,ichimoku_b,ichimoku_base,ichimoku_conv,entry_price,stop_loss,target,trade_outcome
datetime,,,,,,,,,,,,,,,,,
2024-08-08 05:30:00,3185.00,3190.3,3168.00,3169.65,175996.0,3110.137500,3145.3025,3218.758327,3071.846673,3153.1375,3100.275,3130.775,3175.500,3169.65,3106.257,3296.436,No Trigger
2024-08-08 06:00:00,3169.60,3194.6,3162.10,3192.30,259477.0,3112.819318,3150.4725,3221.847198,3079.097802,3153.4375,3100.275,3130.775,3176.100,3192.30,3128.454,3319.992,No Trigger
2024-08-08 06:30:00,3192.00,3226.4,3186.15,3206.00,286073.0,3114.673864,3156.6200,3224.815912,3088.424088,3164.8625,3111.700,3142.200,3187.525,3206.00,3141.880,3334.240,No Trigger
2024-08-08 07:00:00,3206.65,3219.8,3196.80,3199.20,91860.0,3116.903409,3162.6675,3222.988585,3102.346415,3166.4500,3111.700,3142.200,3190.700,3199.20,3135.216,3327.168,No Trigger
2024-08-08 07:30:00,3198.10,3212.8,3193.65,3202.00,65450.0,3120.231818,3169.1325,3215.630105,3122.634895,3168.2250,3111.700,3142.200,3194.250,3202.00,3137.960,3330.080,No Trigger
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-08-04 09:30:00,2362.00,2372.0,2361.60,2371.40,19205.0,2433.820455,2372.8150,2417.427701,2328.202299,2372.4750,2443.700,2389.800,2355.150,2371.40,2323.972,2466.256,None
2025-08-04 10:00:00,2371.30,2374.7,2362.20,2362.40,38661.0,2429.793182,2369.8400,2408.501578,2331.178422,2375.1250,2443.700,2389.800,2360.450,2362.40,2315.152,2456.896,None
2025-08-04 10:30:00,2362.00,2368.7,2358.40,2364.00,42353.0,2425.759091,2367.4600,2401.076984,2333.843016,2375.1250,2443.700,2389.800,2360.450,2364.00,2316.720,2458.560,None


In [3]:
import pandas as pd
from ta.trend import IchimokuIndicator
from ta.volatility import BollingerBands

# ===== Step 1: Load and Prepare Data =====

file_path = '/home/justdial/Bob_Old_Data/Trading_bot/stock_data/Adani enterprise 30 min.xlsx'

raw_df = pd.read_excel(file_path, header=None)
header_row = raw_df[raw_df[0] == 'Local Date'].index[0]
df = pd.read_excel(file_path, header=header_row)

df['datetime'] = pd.to_datetime(df['Local Date'], errors='coerce')
df.dropna(subset=['datetime'], inplace=True)
df.set_index('datetime', inplace=True)

df.rename(columns={
    'Open': 'open', 'High': 'high', 'Low': 'low',
    'Close': 'close', 'Volume': 'volume'
}, inplace=True)

for col in ['open', 'high', 'low', 'close', 'volume']:
    df[col] = pd.to_numeric(df[col], errors='coerce')

df.dropna(subset=['open', 'high', 'low', 'close', 'volume'], inplace=True)
df.sort_index(inplace=True)

# Keep only necessary columns and convert to numeric
expected_cols = ['open', 'high', 'low', 'close', 'volume']
df = df[expected_cols].apply(pd.to_numeric, errors='coerce')
df.dropna(inplace=True)  # Drop any rows with missing price data


# ===== Step 2: Technical Indicators =====

df['MA44'] = df['close'].rolling(window=44).mean()

bb = BollingerBands(close=df['close'], window=20, window_dev=2)
df['bb_bbm'] = bb.bollinger_mavg()
df['bb_bbh'] = bb.bollinger_hband()
df['bb_bbl'] = bb.bollinger_lband()

ich = IchimokuIndicator(high=df['high'], low=df['low'])
df['ichimoku_a'] = ich.ichimoku_a()
df['ichimoku_b'] = ich.ichimoku_b()
df['ichimoku_base'] = ich.ichimoku_base_line()
df['ichimoku_conv'] = ich.ichimoku_conversion_line()

df.dropna(subset=[
    'MA44', 'bb_bbm', 'bb_bbh', 'bb_bbl',
    'ichimoku_a', 'ichimoku_b', 'ichimoku_base', 'ichimoku_conv'
], inplace=True)

# ===== Step 3: Trade Direction Logic =====
# Simple strategy:
# Long if close > MA44 and conversion > base (bullish Ichimoku)
# Short if close < MA44 and conversion < base (bearish Ichimoku)

def get_trade_direction(row):
    if row['close'] > row['MA44'] and row['ichimoku_conv'] > row['ichimoku_base']:
        return 'Long'
    elif row['close'] < row['MA44'] and row['ichimoku_conv'] < row['ichimoku_base']:
        return 'Short'
    else:
        return None

df['trade_type'] = df.apply(get_trade_direction, axis=1)
df['signal'] = df['trade_type'].map({'Long': 'Buy', 'Short': 'Sell'})

# ===== Step 4: Stop Loss & Target Strategy =====

df['entry_price'] = df['close']
risk_pct = 0.02
reward_pct = 0.04

# Different SL/TP for Long and Short trades
df['stop_loss'] = df.apply(lambda r: r['entry_price'] * (1 - risk_pct) if r['trade_type'] == 'Long'
                           else r['entry_price'] * (1 + risk_pct) if r['trade_type'] == 'Short'
                           else None, axis=1)

df['target'] = df.apply(lambda r: r['entry_price'] * (1 + reward_pct) if r['trade_type'] == 'Long'
                        else r['entry_price'] * (1 - reward_pct) if r['trade_type'] == 'Short'
                        else None, axis=1)

# ===== Step 5: Trade Outcome Simulation =====

def simulate_trade_outcome(df, lookahead=5):
    outcomes = []
    profits = []
    for i in range(len(df) - lookahead):
        trade_type = df['trade_type'].iloc[i]
        if not trade_type:
            outcomes.append(None)
            profits.append(None)
            continue

        entry = df['entry_price'].iloc[i]
        stop_loss = df['stop_loss'].iloc[i]
        target = df['target'].iloc[i]
        lows = df['low'].iloc[i+1:i+1+lookahead]
        highs = df['high'].iloc[i+1:i+1+lookahead]

        if trade_type == 'Long':
            sl_hit = lows.min() <= stop_loss
            tp_hit = highs.max() >= target
        else:  # Short
            sl_hit = highs.max() >= stop_loss
            tp_hit = lows.min() <= target

        if sl_hit and tp_hit:
            outcomes.append('Both Hit')
            profits.append(((target - entry) / entry) * 100 if tp_hit else ((stop_loss - entry) / entry) * 100)
        elif sl_hit:
            outcomes.append('Stop Loss Hit')
            profits.append(((stop_loss - entry) / entry) * 100 if trade_type == 'Long' else ((entry - stop_loss) / entry) * 100)
        elif tp_hit:
            outcomes.append('Target Hit')
            profits.append(((target - entry) / entry) * 100 if trade_type == 'Long' else ((entry - target) / entry) * 100)
        else:
            outcomes.append('No Trigger')
            profits.append(0)

    outcomes += [None] * lookahead
    profits += [None] * lookahead
    return outcomes, profits

df['trade_outcome'], df['profit_pct'] = simulate_trade_outcome(df)

# ===== Step 6: Output =====

print("\nTrade Outcome Counts:")
print(df['trade_outcome'].value_counts(dropna=True))

print("\nSample Data:")
print(df[['trade_type', 'signal', 'entry_price', 'stop_loss', 'target', 'trade_outcome', 'profit_pct']].tail(15))



Trade Outcome Counts:
trade_outcome
No Trigger       2484
Stop Loss Hit     214
Target Hit         38
Both Hit            3
Name: count, dtype: int64

Sample Data:
                    trade_type signal  entry_price  stop_loss    target  \
datetime                                                                  
2025-08-01 11:30:00      Short   Sell       2350.9   2397.918  2256.864   
2025-08-04 05:00:00      Short   Sell       2357.9   2405.058  2263.584   
2025-08-04 05:30:00      Short   Sell       2348.9   2395.878  2254.944   
2025-08-04 06:00:00      Short   Sell       2356.8   2403.936  2262.528   
2025-08-04 06:30:00      Short   Sell       2347.9   2394.858  2253.984   
2025-08-04 07:00:00      Short   Sell       2357.5   2404.650  2263.200   
2025-08-04 07:30:00      Short   Sell       2359.1   2406.282  2264.736   
2025-08-04 08:00:00      Short   Sell       2367.5   2414.850  2272.800   
2025-08-04 08:30:00      Short   Sell       2366.7   2414.034  2272.032   
2025-08-04

In [4]:
df

,open,high,low,close,volume,MA44,bb_bbm,bb_bbh,bb_bbl,ichimoku_a,ichimoku_b,ichimoku_base,ichimoku_conv,trade_type,signal,entry_price,stop_loss,target,trade_outcome,profit_pct
datetime,,,,,,,,,,,,,,,,,,,,
2024-08-08 05:30:00,3185.00,3190.3,3168.00,3169.65,175996.0,3110.137500,3145.3025,3218.758327,3071.846673,3153.1375,3100.275,3130.775,3175.500,Long,Buy,3169.65,3106.257,3296.436,No Trigger,0.0
2024-08-08 06:00:00,3169.60,3194.6,3162.10,3192.30,259477.0,3112.819318,3150.4725,3221.847198,3079.097802,3153.4375,3100.275,3130.775,3176.100,Long,Buy,3192.30,3128.454,3319.992,No Trigger,0.0
2024-08-08 06:30:00,3192.00,3226.4,3186.15,3206.00,286073.0,3114.673864,3156.6200,3224.815912,3088.424088,3164.8625,3111.700,3142.200,3187.525,Long,Buy,3206.00,3141.880,3334.240,No Trigger,0.0
2024-08-08 07:00:00,3206.65,3219.8,3196.80,3199.20,91860.0,3116.903409,3162.6675,3222.988585,3102.346415,3166.4500,3111.700,3142.200,3190.700,Long,Buy,3199.20,3135.216,3327.168,No Trigger,0.0
2024-08-08 07:30:00,3198.10,3212.8,3193.65,3202.00,65450.0,3120.231818,3169.1325,3215.630105,3122.634895,3168.2250,3111.700,3142.200,3194.250,Long,Buy,3202.00,3137.960,3330.080,No Trigger,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-08-04 09:30:00,2362.00,2372.0,2361.60,2371.40,19205.0,2433.820455,2372.8150,2417.427701,2328.202299,2372.4750,2443.700,2389.800,2355.150,Short,Sell,2371.40,2418.828,2276.544,None,NaN
2025-08-04 10:00:00,2371.30,2374.7,2362.20,2362.40,38661.0,2429.793182,2369.8400,2408.501578,2331.178422,2375.1250,2443.700,2389.800,2360.450,Short,Sell,2362.40,2409.648,2267.904,None,NaN
2025-08-04 10:30:00,2362.00,2368.7,2358.40,2364.00,42353.0,2425.759091,2367.4600,2401.076984,2333.843016,2375.1250,2443.700,2389.800,2360.450,Short,Sell,2364.00,2411.280,2269.440,None,NaN


In [7]:
import pandas as pd
import numpy as np
from math import floor
from ta.trend import IchimokuIndicator
from ta.volatility import BollingerBands

# ===== Step 1: Load and Prepare Data =====
file_path = '/home/justdial/Bob_Old_Data/Trading_bot/stock_data/Adani enterprise 30 min.xlsx'

raw_df = pd.read_excel(file_path, header=None)
header_row = raw_df[raw_df[0] == 'Local Date'].index[0]
df = pd.read_excel(file_path, header=header_row)

df['datetime'] = pd.to_datetime(df['Local Date'], errors='coerce')
df.dropna(subset=['datetime'], inplace=True)
df.set_index('datetime', inplace=True)

df.rename(columns={
    'Open': 'open', 'High': 'high', 'Low': 'low',
    'Close': 'close', 'Volume': 'volume'
}, inplace=True)

df = df.apply(pd.to_numeric, errors='coerce')
df.dropna(subset=['open', 'high', 'low', 'close', 'volume'], inplace=True)
df.sort_index(inplace=True)

# ===== Step 2: Technical Indicators =====
df['MA44'] = df['close'].rolling(window=44).mean()

bb = BollingerBands(close=df['close'], window=20, window_dev=2)
df['bb_bbm'] = bb.bollinger_mavg()
df['bb_bbh'] = bb.bollinger_hband()
df['bb_bbl'] = bb.bollinger_lband()

ich = IchimokuIndicator(high=df['high'], low=df['low'])
df['ichimoku_a'] = ich.ichimoku_a()
df['ichimoku_b'] = ich.ichimoku_b()
df['ichimoku_base'] = ich.ichimoku_base_line()
df['ichimoku_conv'] = ich.ichimoku_conversion_line()

df.dropna(inplace=True)

# ===== Step 3: Trade Direction Logic =====
mask_long = (df['close'] > df['MA44']) & (df['ichimoku_conv'] > df['ichimoku_base'])
mask_short = (df['close'] < df['MA44']) & (df['ichimoku_conv'] < df['ichimoku_base'])

df['trade_type'] = np.where(mask_long, 'Long', np.where(mask_short, 'Short', None))
df['signal'] = df['trade_type'].map({'Long': 'Buy', 'Short': 'Sell'})

df['reasoning'] = np.where(
    mask_long,
    'Bullish: close > MA44 and Ichimoku conv > base',
    np.where(mask_short,
             'Bearish: close < MA44 and Ichimoku conv < base',
             'No clear Long/Short: conditions not met')
)

# ===== Step 4: Stop Loss & Target =====
risk_pct = 0.02
reward_pct = 0.04

df['entry_price'] = df['close']
df['stop_loss'] = np.where(
    df['trade_type'] == 'Long',
    df['entry_price'] * (1 - risk_pct),
    np.where(df['trade_type'] == 'Short', df['entry_price'] * (1 + risk_pct), np.nan)
)
df['target'] = np.where(
    df['trade_type'] == 'Long',
    df['entry_price'] * (1 + reward_pct),
    np.where(df['trade_type'] == 'Short', df['entry_price'] * (1 - reward_pct), np.nan)
)

# ===== Step 5: Fast Paper Trade Simulator =====
def simulate_paper_trades_fast(df, lookahead=5, principal_inr=10_000_000_000):
    times = df.index.to_numpy()
    entry_prices = df['entry_price'].to_numpy()
    lows = df['low'].to_numpy()
    highs = df['high'].to_numpy()
    trade_types = df['trade_type'].to_numpy()
    stops = df['stop_loss'].to_numpy()
    targets = df['target'].to_numpy()
    reasons = df['reasoning'].to_numpy()

    n = len(df)
    cash = float(principal_inr)
    equity_times = []
    trades = []

    trade_outcomes = np.full(n, None, dtype=object)
    profit_pct = np.full(n, None, dtype=object)

    i = 0
    while i < n:
        equity_times.append((times[i], cash))

        tt = trade_types[i]
        if tt is None:
            i += 1
            continue

        entry = entry_prices[i]
        if not np.isfinite(entry) or entry <= 0:
            i += 1
            continue

        stop_price = stops[i]
        target_price = targets[i]

        shares = floor(cash / entry)
        if shares <= 0:
            i += 1
            continue

        exit_price, exit_idx, triggered = None, None, 'No Trigger'
        end_j = min(i + lookahead, n - 1)

        for j in range(i + 1, end_j + 1):
            if tt == 'Long':
                if highs[j] >= target_price:
                    exit_price, exit_idx, triggered = target_price, j, 'Target Hit'
                    break
                if lows[j] <= stop_price:
                    exit_price, exit_idx, triggered = stop_price, j, 'Stop Loss Hit'
                    break
            else:
                if lows[j] <= target_price:
                    exit_price, exit_idx, triggered = target_price, j, 'Target Hit'
                    break
                if highs[j] >= stop_price:
                    exit_price, exit_idx, triggered = stop_price, j, 'Stop Loss Hit'
                    break

        if exit_price is None:
            exit_idx, exit_price, triggered = end_j, entry_prices[end_j], 'No Trigger'

        pnl = shares * (exit_price - entry) if tt == 'Long' else shares * (entry - exit_price)
        cash += pnl

        trades.append({
            'entry_time': times[i],
            'exit_time': times[exit_idx],
            'trade_type': tt,
            'entry_price': entry,
            'exit_price': exit_price,
            'shares': shares,
            'pnl': pnl,
            'pnl_pct_on_trade': (pnl / (shares * entry) * 100) if shares else 0,
            'triggered': triggered,
            'reasoning': reasons[i]
        })

        trade_outcomes[i] = triggered
        profit_pct[i] = (pnl / (shares * entry) * 100) if shares else 0
        i = exit_idx + 1

    # Safe handling for empty DataFrame
    if n > 0:
        equity_times.append((times[-1], cash))

    trades_df = pd.DataFrame(trades)
    equity_df = pd.DataFrame(equity_times, columns=['timestamp', 'cash']).set_index('timestamp')
    return trades_df, equity_df, trade_outcomes, profit_pct

# ===== Step 6: Run Simulation =====
PRINCIPAL_INR = 10_000_000_000
trades_df, equity_df, outs, pcts = simulate_paper_trades_fast(df, lookahead=5, principal_inr=PRINCIPAL_INR)

df['trade_outcome'] = outs
df['profit_pct'] = pcts

# ===== Step 7: Performance Stats =====
eod_equity = equity_df['cash'].resample('D').last().ffill()
daily_pct = eod_equity.pct_change().fillna(0) * 100
daily_cum_vs_start = (eod_equity / PRINCIPAL_INR - 1) * 100

print(f"Total trades: {len(trades_df)}")
print(trades_df.head(10))
print("\nFinal Cash:", equity_df['cash'].iloc[-1])
print("\nDaily cumulative % vs start:\n", daily_cum_vs_start.tail())


TypeError: Only valid with DatetimeIndex, TimedeltaIndex or PeriodIndex, but got an instance of 'Index'

In [18]:
import pandas as pd
from ta.trend import IchimokuIndicator
from ta.volatility import BollingerBands

# ===== Step 1: Load and Prepare Data =====

file_path = '/home/justdial/Bob_Old_Data/Trading_bot/stock_data/Adani enterprise 30 min.xlsx'

# Read file, find header, then load actual data
raw_df = pd.read_excel(file_path, header=None)
header_row = raw_df[raw_df[0] == 'Local Date'].index[0]
df = pd.read_excel(file_path, header=header_row)

df['datetime'] = pd.to_datetime(df['Local Date'], errors='coerce')
df.dropna(subset=['datetime'], inplace=True)
df.set_index('datetime', inplace=True)

# Standardize column names
df.rename(columns={
    'Open': 'open', 'High': 'high', 'Low': 'low',
    'Close': 'close', 'Volume': 'volume'
}, inplace=True)

# Cast to numeric and drop bad rows
for col in ['open', 'high', 'low', 'close', 'volume']:
    df[col] = pd.to_numeric(df[col], errors='coerce')
df.dropna(subset=['open', 'high', 'low', 'close', 'volume'], inplace=True)
df.sort_index(inplace=True)

# Keep only necessary columns
expected_cols = ['open', 'high', 'low', 'close', 'volume']
df = df[expected_cols].apply(pd.to_numeric, errors='coerce')
df.dropna(inplace=True)

# ===== Step 2: Technical Indicators =====

df['MA44'] = df['close'].rolling(window=44).mean()

bb = BollingerBands(close=df['close'], window=20, window_dev=2)
df['bb_bbm'] = bb.bollinger_mavg()
df['bb_bbh'] = bb.bollinger_hband()
df['bb_bbl'] = bb.bollinger_lband()

ich = IchimokuIndicator(high=df['high'], low=df['low'])
df['ichimoku_a'] = ich.ichimoku_a()
df['ichimoku_b'] = ich.ichimoku_b()
df['ichimoku_base'] = ich.ichimoku_base_line()
df['ichimoku_conv'] = ich.ichimoku_conversion_line()

df.dropna(subset=[
    'MA44', 'bb_bbm', 'bb_bbh', 'bb_bbl',
    'ichimoku_a', 'ichimoku_b', 'ichimoku_base', 'ichimoku_conv'
], inplace=True)

# ===== Step 3: Trade Direction Logic =====

def get_trade_direction(row):
    if row['close'] > row['MA44'] and row['ichimoku_conv'] > row['ichimoku_base']:
        return 'Long'
    elif row['close'] < row['MA44'] and row['ichimoku_conv'] < row['ichimoku_base']:
        return 'Short'
    else:
        return None

df['trade_type'] = df.apply(get_trade_direction, axis=1)
df['signal'] = df['trade_type'].map({'Long': 'Buy', 'Short': 'Sell'})

def reasoning(row):
    if row['trade_type'] == 'Long':
        return "Close > MA44 and Ichimoku Conversion Line > Base Line (Bullish)"
    elif row['trade_type'] == 'Short':
        return "Close < MA44 and Ichimoku Conversion Line < Base Line (Bearish)"
    else:
        return None

df['reasoning'] = df.apply(reasoning, axis=1)

# ===== Step 4: Stop Loss & Target Strategy =====

df['entry_price'] = df['close']
risk_pct = 0.02
reward_pct = 0.04

df['stop_loss'] = df.apply(lambda r: r['entry_price'] * (1 - risk_pct) if r['trade_type'] == 'Long'
                        else r['entry_price'] * (1 + risk_pct) if r['trade_type'] == 'Short'
                        else None, axis=1)
df['target'] = df.apply(lambda r: r['entry_price'] * (1 + reward_pct) if r['trade_type'] == 'Long'
                     else r['entry_price'] * (1 - reward_pct) if r['trade_type'] == 'Short'
                     else None, axis=1)

# ===== Step 5: Trade Outcome Simulation =====

def simulate_trade_outcome(df, lookahead=30):
    outcomes = []
    profits = []
    for i in range(len(df) - lookahead):
        trade_type = df['trade_type'].iloc[i]
        if not trade_type:
            outcomes.append(None)
            profits.append(None)
            continue

        entry = df['entry_price'].iloc[i]
        stop_loss = df['stop_loss'].iloc[i]
        target = df['target'].iloc[i]
        lows = df['low'].iloc[i+1:i+1+lookahead]
        highs = df['high'].iloc[i+1:i+1+lookahead]
        
        if trade_type == 'Long':
            sl_hit = lows.min() <= stop_loss
            tp_hit = highs.max() >= target
        else:  # Short
            sl_hit = highs.max() >= stop_loss
            tp_hit = lows.min() <= target

        if sl_hit and tp_hit:
            outcomes.append('Both Hit')
            profits.append(((target - entry) / entry) * 100 if tp_hit else ((stop_loss - entry) / entry) * 100)
        elif sl_hit:
            outcomes.append('Stop Loss Hit')
            profits.append(((stop_loss - entry) / entry) * 100 if trade_type == 'Long' else ((entry - stop_loss) / entry) * 100)
        elif tp_hit:
            outcomes.append('Target Hit')
            profits.append(((target - entry) / entry) * 100 if trade_type == 'Long' else ((entry - target) / entry) * 100)
        else:
            outcomes.append('No Trigger')
            profits.append(0)

    outcomes += [None] * lookahead
    profits += [None] * lookahead
    return outcomes, profits

df['trade_outcome'], df['profit_pct'] = simulate_trade_outcome(df)

# ===== Step 6: Paper Trade Simulation =====

initial_capital = 10_000_000_000  # 100000 lakh rupees

df['capital'] = initial_capital
df['portfolio_value'] = initial_capital

for i in range(1, len(df)):
    prev_value = df['portfolio_value'].iloc[i-1]
    profit_pct = df['profit_pct'].iloc[i-1] or 0
    # Update capital by applying last trade's profit
    df['capital'].iloc[i] = prev_value * (1 + profit_pct / 100)
    df['portfolio_value'].iloc[i] = df['capital'].iloc[i]

# Set up for percent change calculations
df['date'] = df.index.date

# Use resample directly on DatetimeIndex!
daily = df.resample('D').last()
daily['profit_pct'] = daily['portfolio_value'].pct_change() * 100

weekly = daily.resample('W').last()
weekly['profit_pct'] = weekly['portfolio_value'].pct_change() * 100

monthly = daily.resample('M').last()
monthly['profit_pct'] = monthly['portfolio_value'].pct_change() * 100

# ===== Step 7: Output =====

print("\nTrade Outcome Counts:")
print(df['trade_outcome'].value_counts(dropna=True))

print("\nSample Data With Reasoning:")
print(df[['trade_type', 'signal', 'entry_price', 'stop_loss', 'target', 'trade_outcome', 'profit_pct', 'reasoning']].tail(15))

print("\n==== Paper Trading Profit Percentages ====")
print("\nDaily Profit Pct (last 10):")
print(daily['profit_pct'].tail(10))

print("\nWeekly Profit Pct:")
print(weekly['profit_pct'])

print("\nMonthly Profit Pct:")
print(monthly['profit_pct'])



/tmp/ipykernel_263861/2935587936.py:146: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  df['capital'].iloc[i] = prev_value * (1 + profit_pct / 100)
/tmp/ipykernel_263861/2935587936.py:146: SettingWithCopyWarning: 
A value is trying to be set 


Trade Outcome Counts:
trade_outcome
No Trigger       1310
Stop Loss Hit     989
Target Hit        366
Both Hit           49
Name: count, dtype: int64

Sample Data With Reasoning:
                    trade_type signal  entry_price  stop_loss    target  \
datetime                                                                  
2025-08-01 11:30:00      Short   Sell       2350.9   2397.918  2256.864   
2025-08-04 05:00:00      Short   Sell       2357.9   2405.058  2263.584   
2025-08-04 05:30:00      Short   Sell       2348.9   2395.878  2254.944   
2025-08-04 06:00:00      Short   Sell       2356.8   2403.936  2262.528   
2025-08-04 06:30:00      Short   Sell       2347.9   2394.858  2253.984   
2025-08-04 07:00:00      Short   Sell       2357.5   2404.650  2263.200   
2025-08-04 07:30:00      Short   Sell       2359.1   2406.282  2264.736   
2025-08-04 08:00:00      Short   Sell       2367.5   2414.850  2272.800   
2025-08-04 08:30:00      Short   Sell       2366.7   2414.034  2272.03

/tmp/ipykernel_263861/2935587936.py:154: FutureWarning: The default fill_method='pad' in Series.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  daily['profit_pct'] = daily['portfolio_value'].pct_change() * 100
/tmp/ipykernel_263861/2935587936.py:157: FutureWarning: The default fill_method='pad' in Series.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  weekly['profit_pct'] = weekly['portfolio_value'].pct_change() * 100
/tmp/ipykernel_263861/2935587936.py:159: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  monthly = daily.resample('M').last()
/tmp/ipykernel_263861/2935587936.py:160: FutureWarning: The default fill_method='pad' in Series.pct_change is depre

In [12]:
pip install statsmodels

Defaulting to user installation because normal site-packages is not writeable
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.7/10.7 MB 4.9 MB/s  0:00:02 eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [statsmodels] [statsmodels]
Note: you may need to restart the kernel to use updated packages.


In [22]:
import pandas as pd
import numpy as np
from ta.trend import IchimokuIndicator
from ta.volatility import BollingerBands, AverageTrueRange
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import TimeSeriesSplit
from statsmodels.tsa.arima.model import ARIMA

# Load and prepare data (similar to before)
file_path = '/home/justdial/Bob_Old_Data/Trading_bot/stock_data/Adani enterprise 30 min.xlsx'
raw_df = pd.read_excel(file_path, header=None)
header_row = raw_df[raw_df[0] == 'Local Date'].index[0]
df = pd.read_excel(file_path, header=header_row)
df['datetime'] = pd.to_datetime(df['Local Date'], errors='coerce')
df.dropna(subset=['datetime'], inplace=True)
df.set_index('datetime', inplace=True)
df.rename(columns={
    'Open': 'open', 'High': 'high', 'Low': 'low',
    'Close': 'close', 'Volume': 'volume'
}, inplace=True)
for col in ['open', 'high', 'low', 'close', 'volume']:
    df[col] = pd.to_numeric(df[col], errors='coerce')
df.dropna(subset=['open', 'high', 'low', 'close', 'volume'], inplace=True)
df.sort_index(inplace=True)

# --- All indicators first ---
# df['MA44'] = df['close'].rolling(44).mean()
# bb = BollingerBands(df['close'], window=20, window_dev=2)
# df['bb_bbm'] = bb.bollinger_mavg()
# df['bb_bbh'] = bb.bollinger_hband()
# df['bb_bbl'] = bb.bollinger_lband()
ich = IchimokuIndicator(df['high'], df['low'])
df['ichimoku_a'] = ich.ichimoku_a()
df['ichimoku_b'] = ich.ichimoku_b()
df['ichimoku_base'] = ich.ichimoku_base_line()
df['ichimoku_conv'] = ich.ichimoku_conversion_line()
atr = AverageTrueRange(df['high'], df['low'], df['close'], window=14)
df['atr'] = atr.average_true_range()

# --- ML target ---
df['future_return'] = df['close'].shift(-1) / df['close'] - 1

# --- Time-series forecast target ---
df['log_return'] = np.log(df['close']).diff()

# --- Drop rows with ANY missing values across features, target, forecasting cols ---
df.dropna(subset=[
    'ichimoku_a', 'ichimoku_b', 'ichimoku_base', 'ichimoku_conv',
    'atr', 'future_return', 'log_return'
], inplace=True)

# --- Feature/target sets ---
features = ['ichimoku_a','ichimoku_b','ichimoku_base','ichimoku_conv','atr']
X = df[features]
y = df['future_return']

print("Number of samples before splitting:", len(X))
print("Number of features:", X.shape[1])
print("Any NaNs in X?", X.isna().any().any())
print("Any NaNs in y?", y.isna().any())

# Train Random Forest using time series split
tscv = TimeSeriesSplit(n_splits=5)
model = RandomForestRegressor(n_estimators=100, random_state=42)


for train_index, test_index in tscv.split(X):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
model.fit(X_train, y_train)
df.loc[X_test.index, 'rf_predicted_return'] = model.predict(X_test)

# Time Series Forecasting with ARIMA on close price log returns
df['log_return'] = np.log(df['close']).diff()
df.dropna(subset=['log_return'], inplace=True)
model_arima = ARIMA(df['log_return'], order=(5,0,0))  # AR(5) model
arima_result = model_arima.fit()
df['arima_forecast'] = arima_result.predict(start=5, end=len(df)-1)

# Combine Technical, RF, and ARIMA signals
def combined_signal(row):
    bullish_tech = (row['ichimoku_conv'] > row['ichimoku_base'])
    bearish_tech = (row['ichimoku_conv'] < row['ichimoku_base'])

    rf_pred = row.get('rf_predicted_return', 0)
    arima_pred = row.get('arima_forecast', 0)

    rf_thresh = 0.002
    arima_thresh = 0

    # Enter long if technical bullish AND both forecasts positive over thresholds
    if bullish_tech and rf_pred > rf_thresh and arima_pred > arima_thresh:
        return 'Long'
    # Enter short if technical bearish AND both forecasts negative below thresholds
    elif bearish_tech and rf_pred < -rf_thresh and arima_pred < arima_thresh:
        return 'Short'
    else:
        return None

df['trade_type'] = df.apply(combined_signal, axis=1)



# Define stop loss and target based on ATR
risk_factor = 1.0
reward_factor = 2.0
df['entry_price'] = df['close']
df['stop_loss'] = df.apply(lambda r: r['entry_price'] - risk_factor * r['atr'] if r['trade_type'] == 'Long'
                          else r['entry_price'] + risk_factor * r['atr'] if r['trade_type'] == 'Short'
                          else None, axis=1)
df['target'] = df.apply(lambda r: r['entry_price'] + reward_factor * r['atr'] if r['trade_type'] == 'Long'
                        else r['entry_price'] - reward_factor * r['atr'] if r['trade_type'] == 'Short'
                        else None, axis=1)

# Simulate trade outcomes
def simulate_trade_outcome(df, lookahead=20):
    outcomes = []
    profits = []
    for i in range(len(df) - lookahead):
        trade_type = df['trade_type'].iloc[i]
        if not trade_type:
            outcomes.append(None)
            profits.append(None)
            continue

        entry = df['entry_price'].iloc[i]
        stop_loss = df['stop_loss'].iloc[i]
        target = df['target'].iloc[i]
        lows = df['low'].iloc[i+1:i+1+lookahead]
        highs = df['high'].iloc[i+1:i+1+lookahead]

        if trade_type == 'Long':
            sl_hit = (lows <= stop_loss).any()
            tp_hit = (highs >= target).any()
        else:  # Short
            sl_hit = (highs >= stop_loss).any()
            tp_hit = (lows <= target).any()

        if sl_hit and tp_hit:
            outcomes.append('Both Hit')
            profits.append(((target - entry) / entry) * 100)
        elif sl_hit:
            outcomes.append('Stop Loss Hit')
            profits.append(((stop_loss - entry) / entry) * 100 if trade_type == 'Long' else ((entry - stop_loss) / entry) * 100)
        elif tp_hit:
            outcomes.append('Target Hit')
            profits.append(((target - entry) / entry) * 100 if trade_type == 'Long' else ((entry - target) / entry) * 100)
        else:
            outcomes.append('No Trigger')
            profits.append(0)

    outcomes += [None] * lookahead
    profits += [None] * lookahead
    return outcomes, profits

df['trade_outcome'], df['profit_pct'] = simulate_trade_outcome(df, lookahead=20)

# Paper trading with capital compounding
initial_capital = 10_000_000_000
capital = initial_capital
equity_curve = []
for _, row in df.iterrows():
    profit = row['profit_pct']
    if profit is not None:
        capital *= (1 + profit / 100)
    equity_curve.append(capital)
df['portfolio_value'] = equity_curve

# Print results and resample portfolio for periodic returns
print("Trade counts:")
print(df['trade_outcome'].value_counts(dropna=True))

daily = df.resample('D').last()
daily['daily_return_pct'] = daily['portfolio_value'].pct_change() * 100
weekly = daily.resample('W').last()
weekly['weekly_return_pct'] = weekly['portfolio_value'].pct_change() * 100
monthly = daily.resample('M').last()
monthly['monthly_return_pct'] = monthly['portfolio_value'].pct_change() * 100

print("\nLatest daily returns (%)")
print(daily['daily_return_pct'].tail())

print("\nLatest weekly returns (%)")
print(weekly['weekly_return_pct'].tail())

print("\nLatest monthly returns (%)")
print(monthly['monthly_return_pct'].tail())


Number of samples before splitting: 3464
Number of features: 5
Any NaNs in X? False
Any NaNs in y? False


/home/justdial/.local/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/home/justdial/.local/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/home/justdial/.local/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)


Trade counts:
trade_outcome
Stop Loss Hit    65
Both Hit         43
Target Hit       35
Name: count, dtype: int64

Latest daily returns (%)
datetime
2025-07-31   NaN
2025-08-01   NaN
2025-08-02   NaN
2025-08-03   NaN
2025-08-04   NaN
Freq: D, Name: daily_return_pct, dtype: float64

Latest weekly returns (%)
datetime
2025-07-13   NaN
2025-07-20   NaN
2025-07-27   NaN
2025-08-03   NaN
2025-08-10   NaN
Freq: W-SUN, Name: weekly_return_pct, dtype: float64

Latest monthly returns (%)
datetime
2025-04-30   NaN
2025-05-31   NaN
2025-06-30   NaN
2025-07-31   NaN
2025-08-31   NaN
Freq: ME, Name: monthly_return_pct, dtype: float64


/tmp/ipykernel_263861/3924403903.py:174: FutureWarning: The default fill_method='pad' in Series.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  daily['daily_return_pct'] = daily['portfolio_value'].pct_change() * 100
/tmp/ipykernel_263861/3924403903.py:176: FutureWarning: The default fill_method='pad' in Series.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  weekly['weekly_return_pct'] = weekly['portfolio_value'].pct_change() * 100
/tmp/ipykernel_263861/3924403903.py:177: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  monthly = daily.resample('M').last()
/tmp/ipykernel_263861/3924403903.py:178: FutureWarning: The default fill_method='pad' in Series.pct_ch

In [ ]:
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np
from ta.trend import IchimokuIndicator
from ta.volatility import AverageTrueRange
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import TimeSeriesSplit
from sklearn.base import clone
from statsmodels.tsa.arima.model import ARIMA

# ==== 1) Load Data ====
file_path = '/home/justdial/Bob_Old_Data/Trading_bot/stock_data/Adani enterprise 30 min.xlsx'
raw_df = pd.read_excel(file_path, header=None)
header_row = raw_df[raw_df[0] == 'Local Date'].index[0]
df = pd.read_excel(file_path, header=header_row)
df['datetime'] = pd.to_datetime(df['Local Date'], errors='coerce')
df.dropna(subset=['datetime'], inplace=True)
df.set_index('datetime', inplace=True)
df = df.asfreq('30min')  # enforce uniform spacing
df.rename(columns={
    'Open': 'open', 'High': 'high', 'Low': 'low',
    'Close': 'close', 'Volume': 'volume'
}, inplace=True)
for col in ['open', 'high', 'low', 'close', 'volume']:
    df[col] = pd.to_numeric(df[col], errors='coerce')
df.dropna(subset=['open', 'high', 'low', 'close', 'volume'], inplace=True)
df.sort_index(inplace=True)


# ==== 2) Indicators ====
ich = IchimokuIndicator(df['high'], df['low'])
df['ichimoku_a'] = ich.ichimoku_a()
df['ichimoku_b'] = ich.ichimoku_b()
df['ichimoku_base'] = ich.ichimoku_base_line()
df['ichimoku_conv'] = ich.ichimoku_conversion_line()

atr = AverageTrueRange(df['high'], df['low'], df['close'], window=14)
df['atr'] = atr.average_true_range()

df['future_return'] = df['close'].shift(-1) / df['close'] - 1
df['log_return'] = np.log(df['close']).diff()

req_cols = ['ichimoku_a','ichimoku_b','ichimoku_base','ichimoku_conv','atr','future_return','log_return']
df.dropna(subset=req_cols, inplace=True)

# ==== 3) Walk-forward RF ====
features = ['ichimoku_a','ichimoku_b','ichimoku_base','ichimoku_conv','atr']
X, y = df[features], df['future_return']
tscv = TimeSeriesSplit(n_splits=5)
rf_model = RandomForestRegressor(n_estimators=300, min_samples_leaf=3, random_state=42, n_jobs=-1)
df['rf_predicted_return'] = np.nan
for train_idx, test_idx in tscv.split(X):
    X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
    y_train = y.iloc[train_idx]
    model = clone(rf_model)
    model.fit(X_train, y_train)
    df.loc[X_test.index, 'rf_predicted_return'] = model.predict(X_test)
df['rf_predicted_return'] = df['rf_predicted_return'].rolling(3, min_periods=1).mean()

# ==== 4) Rolling AR Forecast ====
df['arima_forecast'] = np.nan
p, min_fit = 5, 110
logret = df['log_return']
for i in range(min_fit, len(logret)):
    train_series = logret.iloc[:i].dropna()
    freq = pd.infer_freq(train_series.index)
    if freq:  # Only set if inference works
        train_series.index.freq = freq
    try:
        ar = ARIMA(train_series, order=(p,0,0)).fit(method_kwargs={'warn_convergence': False})
        f = ar.forecast(steps=1).iloc[0]
        df.iloc[i, df.columns.get_loc('arima_forecast')] = f
    except:
        continue
df['arima_ret_proxy'] = np.expm1(df['arima_forecast'])

# ==== 5) Signals ====
rf_thresh, arima_thresh = 0.002, 0.0
mask_ok = df['rf_predicted_return'].notna() & df['arima_ret_proxy'].notna()
df['trade_signal'] = None
bull = df['ichimoku_conv'] > df['ichimoku_base']
bear = df['ichimoku_conv'] < df['ichimoku_base']
df.loc[mask_ok & bull & (df['rf_predicted_return'] > rf_thresh) & (df['arima_ret_proxy'] > arima_thresh), 'trade_signal'] = 'Long'
df.loc[mask_ok & bear & (df['rf_predicted_return'] < -rf_thresh) & (df['arima_ret_proxy'] < arima_thresh), 'trade_signal'] = 'Short'

# Entry at next bar open
df['entry_price'] = df['open'].shift(-1)
df['atr_shift1'] = df['atr'].shift(1)
df.dropna(subset=['entry_price'], inplace=True)

# SL/TP
risk_factor, reward_factor = 1.0, 2.0
df['stop_loss'] = np.where(df['trade_signal']=='Long', df['entry_price'] - risk_factor*df['atr_shift1'],
                    np.where(df['trade_signal']=='Short', df['entry_price'] + risk_factor*df['atr_shift1'], np.nan))
df['target'] = np.where(df['trade_signal']=='Long', df['entry_price'] + reward_factor*df['atr_shift1'],
                 np.where(df['trade_signal']=='Short', df['entry_price'] - reward_factor*df['atr_shift1'], np.nan))

# ==== 6) Trade simulation ====
def evaluate_outcome(trade_type, entry, sl, tp, low, high):
    if trade_type == 'Long':
        if (low <= sl) and (high >= tp):
            return 'Both Hit (SL first)', (sl-entry)/entry
        if low <= sl:
            return 'Stop Loss Hit', (sl-entry)/entry
        if high >= tp:
            return 'Target Hit', (tp-entry)/entry
    else:
        if (high >= sl) and (low <= tp):
            return 'Both Hit (SL first)', (entry-sl)/entry
        if high >= sl:
            return 'Stop Loss Hit', (entry-sl)/entry
        if low <= tp:
            return 'Target Hit', (entry-tp)/entry
    return None, 0.0

def simulate_trades(df, lookahead=20):
    outcomes, profits = [None]*len(df), [None]*len(df)
    in_pos = False
    for i in range(len(df)-lookahead):
        sig = df['trade_signal'].iloc[i]
        if not in_pos and sig in ('Long','Short'):
            entry, sl, tp = df['entry_price'].iloc[i], df['stop_loss'].iloc[i], df['target'].iloc[i]
            for k in range(1, lookahead+1):
                o, p = evaluate_outcome(sig, entry, sl, tp, df['low'].iloc[i+k], df['high'].iloc[i+k])
                if o:
                    outcomes[i], profits[i] = o, p*100
                    break
            if outcomes[i] is None:
                outcomes[i], profits[i] = 'No Trigger', 0.0
            in_pos = False
    return outcomes, profits

df['trade_outcome'], df['profit_pct'] = simulate_trades(df)

# ==== 7) Equity curve ====
initial_capital, transaction_bps = 10_000_000_000, 0.0
capital, equity = initial_capital, []
for pr in df['profit_pct']:
    if pr is not None and not pd.isna(pr):
        pr -= transaction_bps/100
        capital *= (1+pr/100)
    equity.append(capital)
df['portfolio_value'] = equity

# ==== 8) Performance ====
trade_counts = pd.Series([x for x in df['trade_outcome'] if x is not None]).value_counts()
final_capital = df['portfolio_value'].iloc[-1]
abs_profit = final_capital - initial_capital
total_return_pct = (final_capital/initial_capital - 1)*100
days = max(1, (df.index[-1] - df.index[0]).days)
cagr = (final_capital/initial_capital)**(365.25/days) - 1

daily = df['portfolio_value'].resample('D').last().ffill().pct_change()*100
weekly = df['portfolio_value'].resample('W-SUN').last().ffill().pct_change()*100
monthly = df['portfolio_value'].resample('M').last().ffill().pct_change()*100

print("Trade counts:\n", trade_counts)
print(f"\nFinal capital: {final_capital:,.2f}")
print(f"Absolute profit: {abs_profit:,.2f}")
print(f"Total return: {total_return_pct:.2f}%")
print(f"CAGR: {cagr*100:.2f}%")
print("\nLatest daily returns (%)\n", daily.tail())
print("\nLatest weekly returns (%)\n", weekly.tail())
print("\nLatest monthly returns (%)\n", monthly.tail())


/home/justdial/.local/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/home/justdial/.local/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/home/justdial/.local/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/home/justdial/.local/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:837: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  retur

In [ ]:
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np
from ta.trend import IchimokuIndicator
from ta.volatility import AverageTrueRange
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import TimeSeriesSplit
from sklearn.base import clone
from statsmodels.tsa.arima.model import ARIMA

# ==== 1) Load Data ====
file_path = '/home/justdial/Bob_Old_Data/Trading_bot/stock_data/Adani enterprise 30 min.xlsx'
raw_df = pd.read_excel(file_path, header=None)
header_row = raw_df[raw_df[0] == 'Local Date'].index[0]
df = pd.read_excel(file_path, header=header_row)
df['datetime'] = pd.to_datetime(df['Local Date'], errors='coerce')
df.dropna(subset=['datetime'], inplace=True)
df.set_index('datetime', inplace=True)
df = df.asfreq('30min')  # enforce uniform spacing
df.rename(columns={
    'Open': 'open', 'High': 'high', 'Low': 'low',
    'Close': 'close', 'Volume': 'volume'
}, inplace=True)
for col in ['open', 'high', 'low', 'close', 'volume']:
    df[col] = pd.to_numeric(df[col], errors='coerce')
df.dropna(subset=['open', 'high', 'low', 'close', 'volume'], inplace=True)
df.sort_index(inplace=True)


# ==== 2) Indicators ====
ich = IchimokuIndicator(df['high'], df['low'])
df['ichimoku_a'] = ich.ichimoku_a()
df['ichimoku_b'] = ich.ichimoku_b()
df['ichimoku_base'] = ich.ichimoku_base_line()
df['ichimoku_conv'] = ich.ichimoku_conversion_line()

atr = AverageTrueRange(df['high'], df['low'], df['close'], window=14)
df['atr'] = atr.average_true_range()

df['future_return'] = df['close'].shift(-1) / df['close'] - 1
df['log_return'] = np.log(df['close']).diff()

req_cols = ['ichimoku_a','ichimoku_b','ichimoku_base','ichimoku_conv','atr','future_return','log_return']
df.dropna(subset=req_cols, inplace=True)

# ==== 3) Walk-forward RF ====
features = ['ichimoku_a','ichimoku_b','ichimoku_base','ichimoku_conv','atr']
X, y = df[features], df['future_return']
tscv = TimeSeriesSplit(n_splits=5)
rf_model = RandomForestRegressor(n_estimators=300, min_samples_leaf=3, random_state=42, n_jobs=-1)
df['rf_predicted_return'] = np.nan
for train_idx, test_idx in tscv.split(X):
    X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
    y_train = y.iloc[train_idx]
    model = clone(rf_model)
    model.fit(X_train, y_train)
    df.loc[X_test.index, 'rf_predicted_return'] = model.predict(X_test)
df['rf_predicted_return'] = df['rf_predicted_return'].rolling(3, min_periods=1).mean()

# ==== 4) Rolling AR Forecast ====
df['arima_forecast'] = np.nan
p, min_fit = 5, 110
logret = df['log_return']
for i in range(min_fit, len(logret)):
    train_series = logret.iloc[:i].dropna()
    freq = pd.infer_freq(train_series.index)
    if freq:
        train_series.index.freq = freq
    try:
        ar = ARIMA(train_series, order=(p,0,0)).fit(method_kwargs={'warn_convergence': False})
        f = ar.forecast(steps=1).iloc[0]
        df.iloc[i, df.columns.get_loc('arima_forecast')] = f
    except:
        continue
df['arima_ret_proxy'] = np.expm1(df['arima_forecast'])

# ==== 5) Signals ====
rf_thresh, arima_thresh = 0.002, 0.0
mask_ok = df['rf_predicted_return'].notna() & df['arima_ret_proxy'].notna()
df['trade_signal'] = None
bull = df['ichimoku_conv'] > df['ichimoku_base']
bear = df['ichimoku_conv'] < df['ichimoku_base']
df.loc[mask_ok & bull & (df['rf_predicted_return'] > rf_thresh) & (df['arima_ret_proxy'] > arima_thresh), 'trade_signal'] = 'Long'
df.loc[mask_ok & bear & (df['rf_predicted_return'] < -rf_thresh) & (df['arima_ret_proxy'] < arima_thresh), 'trade_signal'] = 'Short'

# Entry at next bar open
df['entry_price'] = df['open'].shift(-1)
df['atr_shift1'] = df['atr'].shift(1)
df.dropna(subset=['entry_price'], inplace=True)

# SL/TP
risk_factor, reward_factor = 1.0, 2.0
df['stop_loss'] = np.where(df['trade_signal']=='Long', df['entry_price'] - risk_factor*df['atr_shift1'],
                    np.where(df['trade_signal']=='Short', df['entry_price'] + risk_factor*df['atr_shift1'], np.nan))
df['target'] = np.where(df['trade_signal']=='Long', df['entry_price'] + reward_factor*df['atr_shift1'],
                 np.where(df['trade_signal']=='Short', df['entry_price'] - reward_factor*df['atr_shift1'], np.nan))

# ==== 6) Trade simulation ====
def evaluate_outcome(trade_type, entry, sl, tp, low, high):
    if trade_type == 'Long':
        if (low <= sl) and (high >= tp):
            return 'Both Hit (SL first)', (sl-entry)/entry
        if low <= sl:
            return 'Stop Loss Hit', (sl-entry)/entry
        if high >= tp:
            return 'Target Hit', (tp-entry)/entry
    else:
        if (high >= sl) and (low <= tp):
            return 'Both Hit (SL first)', (entry-sl)/entry
        if high >= sl:
            return 'Stop Loss Hit', (entry-sl)/entry
        if low <= tp:
            return 'Target Hit', (entry-tp)/entry
    return None, 0.0

def simulate_trades(df, lookahead=20):
    outcomes, profits = [None]*len(df), [None]*len(df)
    in_pos = False
    for i in range(len(df)-lookahead):
        sig = df['trade_signal'].iloc[i]
        if not in_pos and sig in ('Long','Short'):
            entry, sl, tp = df['entry_price'].iloc[i], df['stop_loss'].iloc[i], df['target'].iloc[i]
            for k in range(1, lookahead+1):
                o, p = evaluate_outcome(sig, entry, sl, tp, df['low'].iloc[i+k], df['high'].iloc[i+k])
                if o:
                    outcomes[i], profits[i] = o, p*100
                    break
            if outcomes[i] is None:
                outcomes[i], profits[i] = 'No Trigger', 0.0
            in_pos = False
    return outcomes, profits

df['trade_outcome'], df['profit_pct'] = simulate_trades(df)

# ==== 7) Equity curve ====
initial_capital, transaction_bps = 10_000_000_000, 0.0
capital, equity = initial_capital, []
for pr in df['profit_pct']:
    if pr is not None and not pd.isna(pr):
        pr -= transaction_bps/100
        capital *= (1+pr/100)
    equity.append(capital)
df['portfolio_value'] = equity

# ==== 8) Performance ====
trade_counts = pd.Series([x for x in df['trade_outcome'] if x is not None]).value_counts()
final_capital = df['portfolio_value'].iloc[-1]
abs_profit = final_capital - initial_capital
total_return_pct = (final_capital/initial_capital - 1)*100
days = max(1, (df.index[-1] - df.index[0]).days)
cagr = (final_capital/initial_capital)**(365.25/days) - 1

daily = df['portfolio_value'].resample('D').last().ffill().pct_change()*100
weekly = df['portfolio_value'].resample('W-SUN').last().ffill().pct_change()*100
monthly = df['portfolio_value'].resample('M').last().ffill().pct_change()*100

print("Trade counts:\n", trade_counts)
print(f"\nFinal capital: {final_capital:,.2f}")
print(f"Absolute profit: {abs_profit:,.2f}")
print(f"Total return: {total_return_pct:.2f}%")
print(f"CAGR: {cagr*100:.2f}%")
print("\nLatest daily returns (%)\n", daily.tail())
print("\nLatest weekly returns (%)\n", weekly.tail())
print("\nLatest monthly returns (%)\n", monthly.tail())


/home/justdial/.local/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/home/justdial/.local/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/home/justdial/.local/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/home/justdial/.local/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:837: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  retur